# C3 Style new- table D.1/C.1

In [18]:
pip install pytesseract

  Using cached pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
Using cached pytesseract-0.3.13-py3-none-any.whl (14 kB)
Note: you may need to restart the kernel to use updated packages.


In [19]:
import pytesseract

In [20]:
import os
import pdfplumber
import pandas as pd
import re
import PyPDF2
import traceback

# Define paths and project range
BASE_DIRECTORY ="/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data"
OUTPUT_CSV_PATH_ORIGINAL = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/03_raw/ph2_rawdata_cluster3_style_new_originals.csv"
OUTPUT_CSV_PATH_ADDENDUM = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/03_raw/ph2_rawdata_cluster3_style_new_addendums.csv"
LOG_FILE_PATH = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/03_raw/ph2_scraping_cluster3_style_new_log.txt"
PROJECT_RANGE = range(643, 644)  # Inclusive range for q_ids in Clusters 3 range(667, 860)


# Read the CSV file containing processed projects (with q_id column)
processed_csv_path = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/all_clusters/costs_phase_2_all_clusters_total.csv"  # UPDATE THIS PATH
processed_df = pd.read_csv(processed_csv_path)
# Convert q_id values to numeric then to int for filtering
processed_q_ids = pd.to_numeric(processed_df['q_id'], errors='coerce').dropna().astype(int).unique()
# Now build the list of folders to process:
projects_to_process = []
for folder in os.listdir(BASE_DIRECTORY):
    full = os.path.join(BASE_DIRECTORY, folder)
    if not os.path.isdir(full):
        continue
    m = re.match(r'^(\d+)', folder)
    if not m:
        continue
    num = int(m.group(1))
    if num in PROJECT_RANGE and num not in processed_q_ids:
        projects_to_process.append(folder)

print("Will process these project‐folders:")
print(projects_to_process)

# Initialize DataFrames
core_originals = pd.DataFrame()
core_addendums = pd.DataFrame()

# Initialize tracking variables
scraped_projects = set()
skipped_projects = set()
missing_projects = set()
scraped_pdfs = []
skipped_pdfs = []
addendum_pdfs = []
original_pdfs = []
style_n_pdfs = []  # List to track style N PDFs
total_pdfs_accessed = 0
total_pdfs_scraped = 0
total_pdfs_skipped = 0
total_pdfs_skipped_extraction = 0
original_has_table7 = {}  # Dictionary to track if original PDFs have table7

def clean_column_headers(headers):
    """Cleans column headers by normalizing and removing unwanted characters, but keeps parentheses."""
    cleaned_headers = []
    for header in headers:
        if header is None:
            header = ""
        elif isinstance(header, str):
            header = header.lower()
            # collapse internal whitespace
            header = re.sub(r'\s+', ' ', header)
            # strip out everything except letters, digits, spaces, and parentheses
            header = re.sub(r'[^a-z0-9\s\(\)]', '', header)
            header = header.strip()
        cleaned_headers.append(header)
    return cleaned_headers

def clean_string_cell(value):
    """Cleans string cells by removing newlines and trimming spaces."""
    if isinstance(value, str):
        return value.replace('\n', ' ').strip()
    elif value is None:
        return ""
    else:
        return str(value).replace('\n', ' ').strip()
     

def contains_phrase(row, phrase):
    """Checks if any cell in a row contains a specific phrase."""
    regex_pattern = re.sub(r"\s+", r"\\s*", phrase)
    pattern = re.compile(regex_pattern, flags=re.IGNORECASE)
    return row.astype(str).apply(lambda cell: bool(pattern.search(cell))).any()

def extract_specific_phrase(title):
    """
    Extracts a specific phrase from the table title based on predefined keywords.

    Args:
        title (str): The table title string.

    Returns:
        str: The extracted specific phrase if found, else the original title.
    """
    phrases = [
        "PTO",
        "Reliability Network Upgrade",
        "Area Delivery Network Upgrade",
        "Local Delivery Network",
         
        
        "Other Potential Network Upgrade",
        "Area Delivery Network Upgrades",
        "Conditionally Assigned Network Upgrades",
        "Local Off-Peak Network Upgrade",
        "ADNU",
        "LDNU",
        "RNU"
    ]

    for phrase in phrases:
        if  re.search(rf"\b{re.escape(phrase)}\b(?=\d|\W|$)", title, re.IGNORECASE):
        
         #re.search(rf"\b{re.escape(phrase)}\b", title, re.IGNORECASE):
            return phrase
    return title  # Fallback to the entire title if no specific phrase is found

def reorder_columns(df):
    """
    Reorders the columns of the DataFrame based on the specified order.

    Args:
        df (pd.DataFrame): The DataFrame to reorder.

    Returns:
        pd.DataFrame: The reordered DataFrame.
    """
    desired_order = [
        "q_id",
        "cluster",
        "req_deliverability",
        "latitude",
        "longitude",
        "capacity",
        "point_of_interconnection",
        "type of upgrade",
        "upgrade",
        "description",
        "cost allocation factor"
    ]

    # Start with desired columns that exist in the DataFrame
    existing_desired = [col for col in desired_order if col in df.columns]

    # Then add the remaining columns
    remaining = [col for col in df.columns if col not in existing_desired]

    # Combine the two lists
    new_order = existing_desired + remaining

    # Reorder the DataFrame
    df = df[new_order]

    return df

def search_gps_coordinates(text, log_file):
    """Search for GPS coordinates using multiple patterns."""
    gps_coords = re.search(r"gps coordinates:\s*([\d\.\-]+),\s*([\d\.\-]+)", text, re.IGNORECASE)
    if gps_coords:
        print(f"Found GPS coordinates: {gps_coords.groups()}", file=log_file)
        return gps_coords.groups()

    project_coords = re.search(r"latitude[:\s]*([\d\.\-]+)[^\d]+longitude[:\s]*([\d\.\-]+)", text, re.IGNORECASE)
    if project_coords:
        print(f"Found project coordinates: {project_coords.groups()}", file=log_file)
        return project_coords.groups()

    gps_coords_directional = re.search(
        r"gps coordinates:\s*([\d\.\-]+)\s*[nNsS],\s*([\d\.\-]+)\s*[eEwW]", text, re.IGNORECASE)
    if gps_coords_directional:
        lat, lon = gps_coords_directional.groups()
        latitude = lat if "N" in text.upper() else f"-{lat}"  # Adjust latitude sign
        longitude = lon if "E" in text.upper() else f"-{lon}"  # Adjust longitude sign
        print(f"Found directional GPS coordinates: {(latitude, longitude)}", file=log_file)
        return (latitude, longitude)

    print("GPS coordinates not found.", file=log_file)
    return (None, None)

def extract_table1(pdf_path, log_file):
    """
    Extracts the Point of Interconnection from Table 1 in the provided PDF.
    Implements a retry mechanism with different table extraction settings if initial attempts fail.

    Args:
        pdf_path (str): Path to the PDF file.
        log_file (file object): Log file to write print statements.

    Returns:
        str: Extracted Point of Interconnection value,
             "Value Missing" if label found but no value,
             or None if not found.
    """
    print(f"\nProcessing {pdf_path} for Table 1 extraction...", file=log_file)
    point_of_interconnection = None

    # Define the regex pattern for 'Point of Interconnection' (case-insensitive)
    poi_pattern = re.compile(r"Point\s+of\s+Interconnection", re.IGNORECASE)

    # Define different table extraction settings to try
    table_settings_list = [
        {
            "horizontal_strategy": "text",
            "vertical_strategy": "lines",
            "snap_tolerance": 1,
        },
        {
            "horizontal_strategy": "lines",
            "vertical_strategy": "lines",
            "snap_tolerance": 2,  # Increased tolerance for retry
        }
    ]

    try:
        with pdfplumber.open(pdf_path) as pdf:
            # Identify all pages that contain "Table 1"
            table1_pages = []
            for i, page in enumerate(pdf.pages):
                text = page.extract_text() or ""
                if re.search(r"Table\s*B.1\b", text, re.IGNORECASE):
                    table1_pages.append(i)

            if not table1_pages:
                print("No Table 1 found in the PDF.", file=log_file)
                return None  # Return None if no Table 1 found

            first_page = table1_pages[0]
            last_page = table1_pages[-1]
            scrape_start = first_page
            scrape_end = last_page + 2  # Plus one to include the next page if needed

            print(f"Table 1 starts on page {scrape_start + 1} and ends on page {scrape_end + 1}", file=log_file)

            # Flag to indicate if extraction was successful
            extraction_successful = False

            # Iterate through the specified page range
            for page_number in range(scrape_start, min(scrape_end + 1, len(pdf.pages))):
                page = pdf.pages[page_number]
                print(f"\nScraping tables on page {page_number + 1} for Table 1...", file=log_file)

                for attempt, table_settings in enumerate(table_settings_list, start=1):
                    print(f"\nAttempt {attempt} with table settings: {table_settings}", file=log_file)
                    tables = page.find_tables(table_settings=table_settings)
                    print(f"Found {len(tables)} table(s) on page {page_number + 1} with current settings.", file=log_file)

                    for table_index, table in enumerate(tables, start=1):
                        tab = table.extract()
                        if not tab:
                            print(f"Table {table_index} on page {page_number + 1} is empty. Skipping.", file=log_file)
                            continue  # Skip empty tables

                        print(f"\n--- Table {table_index} on Page {page_number + 1} ---", file=log_file)
                        for row_num, row in enumerate(tab, start=1):
                            print(f"Row {row_num}: {row}", file=log_file)

                        # Iterate through each row in the table
                        for row_index, row in enumerate(tab, start=1):
                            # Iterate through each cell in the row
                            for cell_index, cell in enumerate(row, start=1):
                                if cell and poi_pattern.search(cell):
                                    # Assuming the next column contains the value
                                    poi_col_index = cell_index  # 1-based index
                                    adjacent_col_index = poi_col_index + 1  # Next column

                                    if adjacent_col_index <= len(row):
                                        poi_value = clean_string_cell(row[adjacent_col_index - 1])
                                        if poi_value:  # Check if the value is not empty
                                            point_of_interconnection = poi_value
                                            print(f"\nFound Point of Interconnection: '{point_of_interconnection}' "
                                                  f"(Page {page_number + 1}, Table {table_index}, Row {row_index})", file=log_file)
                                            extraction_successful = True
                                            break  # Exit the cell loop
                                        else:
                                            print(f"\nPoint of Interconnection label found but adjacent value is empty "
                                                  f"(Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                                            # Proceed to scan surrounding rows for the value
                                            poi_value_parts = []

                                            # Define the range to scan: two rows above and two rows below
                                            # Convert to 0-based index
                                            current_row_idx = row_index - 1
                                            start_scan = max(0, current_row_idx - 2)
                                            end_scan = min(len(tab), current_row_idx + 3)  # Exclusive

                                            print(f"Scanning rows {start_scan + 1} to {end_scan} for POI value parts.", file=log_file)

                                            for scan_row_index in range(start_scan, end_scan):
                                                # Skip the current row where the label was found
                                                if scan_row_index == current_row_idx:
                                                    continue

                                                scan_row = tab[scan_row_index]
                                                # Ensure the adjacent column exists in the scan row
                                                if adjacent_col_index - 1 < len(scan_row):
                                                    scan_cell = clean_string_cell(scan_row[adjacent_col_index - 1])
                                                    if scan_cell and not poi_pattern.search(scan_cell):
                                                        poi_value_parts.append(scan_cell)
                                                        print(f"Found POI part in row {scan_row_index + 1}: '{scan_cell}'", file=log_file)
                                                    elif poi_pattern.search(scan_cell):
                                                        # If another POI label is found, skip it
                                                        print(f"Encountered another POI label in row {scan_row_index + 1}. Skipping this row.", file=log_file)
                                                        continue

                                            if poi_value_parts:
                                                # Concatenate the parts to form the complete POI value
                                                point_of_interconnection = " ".join(poi_value_parts)
                                                print(f"\nConcatenated Point of Interconnection: '{point_of_interconnection}' "
                                                      f"(Page {page_number + 1}, Table {table_index})", file=log_file)
                                                extraction_successful = True
                                                break  # Exit the cell loop
                                            else:
                                                print(f"\nNo POI value found in the surrounding rows "
                                                      f"(Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                                                # Do not return immediately; proceed to retry
                                    else:
                                        print(f"\nPoint of Interconnection label found but no adjacent column "
                                              f"(Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                                        # Do not return immediately; proceed to retry
                            if extraction_successful:
                                break  # Exit the row loop
                        if extraction_successful:
                            break  # Exit the table loop
                    if extraction_successful:
                        break  # Exit the attempt loop
                if extraction_successful:
                    break  # Exit the page loop

    except Exception as e:
        print(f"Error processing Table 1 in {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
        return None

    if not extraction_successful:
        # After all attempts, determine the appropriate return value
        if point_of_interconnection is not None and point_of_interconnection != "":
            # Label was found but no value
            print("Point of Interconnection label found but no adjacent value.", file=log_file)
            return "Value Missing"
        else:
            # Label not found
            print("Point of Interconnection not found in Table 1.", file=log_file)
            return None

    return point_of_interconnection

def extract_base_data(pdf_path, project_id, log_file):
    """Extract base data from the PDF and return as a DataFrame."""
    print("Extracting base data from PDF...", file=log_file)
    try:
        with open(pdf_path, 'rb') as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text

        text = clean_string_cell(text)

        queue_id = re.search(r"q[\s_-]*(\d+)", text, re.IGNORECASE)
        queue_id =  str(project_id)  # Use project_id if queue_id is not found
        print(f"Extracted Queue ID: {queue_id}", file=log_file)

        # Updated Cluster Extraction
        clusters = re.findall(r"queue[\s_-]*cluster[\s_-]*(\d+)", text, re.IGNORECASE)
        if '3' in clusters:
            cluster_number = '3'
        elif clusters:
            cluster_number = max(clusters, key=lambda x: int(x))  # Choose the highest cluster number found
        else:
            cluster_number = '3'  # Default to 3 if not found
        print(f"Extracted Cluster Number: {cluster_number}", file=log_file)

        deliverability_status = re.search(r"(\w+)\s*capacity deliverability status", text, re.IGNORECASE)
        deliverability_status = deliverability_status.group(1) if deliverability_status else None
        print(f"Extracted Deliverability Status: {deliverability_status}", file=log_file)

        # Extract Capacity
        capacity = re.search(r"total rated output of (\d+)\s*mw", text, re.IGNORECASE)
        if capacity:
            capacity = int(capacity.group(1))
        else:
            capacity2 = re.search(r"(\d+)\s*mw", text)
            capacity = int(capacity2.group(1)) if capacity2 else None
        print(f"Extracted Capacity: {capacity}", file=log_file)

        # Extract Point of Interconnection
        point_of_interconnection = extract_table1(pdf_path, log_file)

        latitude, longitude = search_gps_coordinates(text, log_file)

        # Initialize base data dictionary
        base_data = {
            "q_id": [queue_id],
            "cluster": [cluster_number],
            "req_deliverability": [deliverability_status],
            "latitude": [latitude],
            "longitude": [longitude],
            "capacity": [capacity],
            "point_of_interconnection": [point_of_interconnection]
        }

        print("Base data extracted:", file=log_file)
        print(base_data, file=log_file)
        return pd.DataFrame(base_data)

    except Exception as e:
        print(f"Error extracting base data from {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
        return pd.DataFrame()  # Return empty DataFrame on error

def adjust_rows_length(data_rows, headers):
    """Ensure each row in data_rows matches the length of headers by truncating or padding."""
    col_count = len(headers)
    for i in range(len(data_rows)):
        row = data_rows[i]
        if len(row) > col_count:
            data_rows[i] = row[:col_count]
        elif len(row) < col_count:
            data_rows[i].extend([""]*(col_count - len(row)))

def extract_table7(pdf_path, log_file, is_addendum=False):
    """
    Extracts Table 3 data from the provided PDF.

    Args:
        pdf_path (str): Path to the PDF file.
        log_file (file object): Log file to write print statements.
        is_addendum (bool): Whether the PDF is an addendum.

    Returns:
        pd.DataFrame: Extracted Table 3 data.
    """
    print(f"\nProcessing {pdf_path} for Table 3 extraction...", file=log_file)
    extracted_tables = []
    specific_phrase = None

    try:
        with pdfplumber.open(pdf_path) as pdf:
            # Identify pages that contain "Table 3-1" to "Table 3-3" with hyphen or dot
            table7_pages = []
            for i, page in enumerate(pdf.pages):
                text = page.extract_text() or ""
                if re.search(r"Table\s*[C-D]\s*[\.-]\s*1\s*:", text, re.IGNORECASE): # the \s* is to match any whitespace between the table number and the colon
                    table7_pages.append(i)

            if not table7_pages:
                print("No Table 3-1 to 3-3 found in the PDF.", file=log_file)
                return pd.DataFrame()

            first_page = table7_pages[0]
            last_page = table7_pages[-1]
            scrape_start = first_page
            scrape_end = last_page + 3  # Plus two to include possible continuation

            print(f"Table 3 starts on page {scrape_start} and ends on page {scrape_end}", file=log_file)

            for page_number in range(scrape_start, min(scrape_end, len(pdf.pages))):
                page = pdf.pages[page_number]
                print(f"\nScraping tables on page {page_number + 1}...", file=log_file)
                tables = page.find_tables(table_settings={
                    "horizontal_strategy": "lines",
                    "vertical_strategy": "lines",
                    
                })

                for table_index, table in enumerate(tables):
                    tab = table.extract()
                    if not tab:
                        print(f"Table {table_index + 1} on page {page_number + 1} is empty. Skipping.", file=log_file)
                        continue

                    table_bbox = table.bbox
                    title_bbox = (0, 0, page.width, table_bbox[1])
                    title_text = page.within_bbox(title_bbox).extract_text() or ""
                    table_title = None

                    if title_text:
                        title_lines = title_text.split('\n')[::-1]
                        for line in title_lines:
                            line = line.strip()
                            match = re.search(r"(Modification\s+of\s+)?Table\s*[C-D][\.-]([1-2])[:\-\s]*(.*)", line, re.IGNORECASE)
                            if match:
                                try:
                                    table_title = match.group(3).strip()
                                except IndexError:
                                    table_title = match.group(0).strip()
                                    print("Fallback to whole match for table title", file=log_file)

 
                


                    if table_title:
                        if re.search(r"PTO Interconnection Facilities Cost Estimate Summary", table_title, re.IGNORECASE):
                            print(f"Skipping Table 8-1 PTO on page {page_number + 1}, table {table_index + 1}.", file=log_file)
                            continue

                    if table_title:
                        if re.search(r"\b8-7\b", table_title, re.IGNORECASE):
                            print(f"Skipping Table 3-3 on page {page_number + 1}, table {table_index + 1}.", file=log_file)
                            continue  # Skip Table 3-3

                        

                        # New Table 3 detected
                        specific_phrase = extract_specific_phrase(table_title)
                        print(f"New Table 3 detected: '{specific_phrase}' on page {page_number + 1}, table {table_index + 1}", file=log_file)

                        headers = clean_column_headers(tab[0])
                        data_rows = tab[1:]

                        # Create DataFrame for new table
                        try:
                            df_new = pd.DataFrame(data_rows, columns=headers)
                        except ValueError as ve:
                            print(f"Error creating DataFrame for new table on page {page_number + 1}, table {table_index + 1}: {ve}", file=log_file)
                            continue

                            # ← INSERT this block:
                        if df_new.empty:
                            # store an empty DF with the right columns,
                            # so that continuation blocks can append to it
                            extracted_tables.append(pd.DataFrame(columns=headers))
                            print(f"Header-only Table 3 (‘{specific_phrase}’) detected on page {page_number+1}; waiting for continuation…", file=log_file)
                            continue

                        # Handle ADNU-specific grouping
                        if re.search(r"Area\s*Delivery\s*Network\s*Upgrade", specific_phrase, re.IGNORECASE):
                            print("Detected 'Area Delivery Network Upgrade' table (new).", file=log_file)
                            if "adnu" in df_new.columns:
                                if "type of upgrade" not in df_new.columns:
                                    # Group all adnu rows into one 'upgrade' row
                                    adnu_values = df_new["adnu"].dropna().astype(str).tolist()
                                    grouped_adnu = " ".join(adnu_values)
                                    other_columns = df_new.drop(columns=["adnu"]).iloc[0].to_dict()

                                    df_grouped = pd.DataFrame({
                                        "upgrade": [grouped_adnu],
                                        "type of upgrade": [specific_phrase]
                                    })

                                    for col, value in other_columns.items():
                                        df_grouped[col] = value

                                    print("Grouped all 'adnu' rows into a single 'upgrade' row for new ADNU table.", file=log_file)
                                    df_new = df_grouped
                                else:
                                    # If 'type of upgrade' exists, just rename adnu if needed
                                    if "upgrade" in df_new.columns:
                                        df_new.drop(columns=['adnu'], inplace=True)
                                        print("Dropped 'adnu' column to avoid duplicate 'upgrade'.", file=log_file)
                                    else:
                                        df_new.rename(columns={'adnu': 'upgrade'}, inplace=True)
                                        print("Renamed 'adnu' to 'upgrade' in new ADNU table.", file=log_file)
                            if "type of upgrade" not in df_new.columns:
                                df_new["type of upgrade"] = specific_phrase
                                print("Added 'type of upgrade' to all rows in new ADNU table.", file=log_file)
                            else:
                                # If 'type of upgrade' exists and first row is none, replace only first row if needed
                                if df_new.empty:
                                    # should never happen once you’ve done step 1, but safe to check
                                    continue
                                first_row = df_new.iloc[0]
                                if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                    df_new.at[0, "type of upgrade"] = specific_phrase
                                    print("Replaced None in 'type of upgrade' first row for new ADNU table.", file=log_file)
                        else:
                            # Non-ADNU new tables
                            if "type of upgrade" not in df_new.columns:
                                df_new["type of upgrade"] = specific_phrase
                                print("Added 'type of upgrade' to all rows in new non-ADNU table.", file=log_file)
                            else:
                                # If 'type of upgrade' exists and first row is none, replace only first row if needed
                                first_row = df_new.iloc[0]
                                if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                    print("Replacing None in 'type of upgrade' for the first row in new non-ADNU table.", file=log_file)
                                    df_new.at[0, "type of upgrade"] = specific_phrase

                        # Ensure no duplicate columns
                        if df_new.columns.duplicated().any():
                            print("Duplicate columns detected in new table. Dropping duplicates.", file=log_file)
                            df_new = df_new.loc[:, ~df_new.columns.duplicated()]

                        extracted_tables.append(df_new)
                    else:
                        # Continuation Table
                        if not extracted_tables:
                            print(f"No previous Table 3 detected to continue with on page {page_number + 1}, table {table_index + 1}. Skipping.", file=log_file)
                            continue

                        last_table = extracted_tables[-1]
                        expected_columns = last_table.columns.tolist()

                        print(f"Continuation Table detected on page {page_number + 1}, table {table_index + 1}", file=log_file)
                        data_rows = tab

                        # Check if the first row is a header row
                        # As per your latest instruction, we will treat all continuation table rows as data points
                        # without any header detection
                        # However, you mentioned checking if there is a header row first, so we'll implement that

                        # Detect if first row is a header
                        header_keywords = ["type of upgrade", "adnu", "MW at POI"]
                        first_row = data_rows[0] if data_rows else []
                        is_header_row = any(
                            any(re.search(rf"\b{kw}\b", clean_string_cell(cell).lower()) for kw in header_keywords)
                            for cell in first_row
                        )

                        if is_header_row:
                            # Handle header row in continuation table
                            headers = clean_column_headers(first_row)
                            data_rows = data_rows[1:]  # Exclude header row

                            # Update expected_columns by adding new columns if any
                            new_columns = [col for col in headers if col not in expected_columns]
                            if new_columns:
                                expected_columns.extend(new_columns)
                                print(f"Added new columns from continuation table: {new_columns}", file=log_file)

                            # Create a mapping of new columns to add with default NaN
                            for new_col in new_columns:
                                last_table[new_col] = pd.NA

                            # Reindex last_table to include new columns
                            last_table = last_table.reindex(columns=expected_columns)
                            extracted_tables[-1] = last_table

                            # Update 'type of upgrade' column in the first row if needed
                            if "type of upgrade" in headers:
                                type_upgrade_idx = headers.index("type of upgrade")
                                if pd.isna(data_rows[0][type_upgrade_idx]) or data_rows[0][type_upgrade_idx] == "":
                                    data_rows[0][type_upgrade_idx] = specific_phrase
                                    print(f"Replaced None in 'type of upgrade' first row for continuation table on page {page_number + 1}, table {table_index + 1}.", file=log_file)
                            elif "upgrade" in headers:
                                upgrade_idx = headers.index("upgrade")
                                if pd.isna(data_rows[0][upgrade_idx]) or data_rows[0][upgrade_idx] == "":
                                    data_rows[0][upgrade_idx] = specific_phrase
                                    print(f"Replaced None in 'upgrade' first row for continuation table on page {page_number + 1}, table {table_index + 1}.", file=log_file)
                            else:
                                # If 'type of upgrade' or 'upgrade' does not exist, add it
                                headers.append("type of upgrade")
                                expected_columns.append("type of upgrade")
                                for idx, row in enumerate(data_rows):
                                    data_rows[idx].append(specific_phrase)
                                print(f"Added 'type of upgrade' column and filled with '{specific_phrase}' for continuation table on page {page_number + 1}, table {table_index + 1}.", file=log_file)

                            # Handle ADNU-specific logic if applicable
                            if re.search(r"Area\s*Delivery\s*Network\s*Upgrade", specific_phrase, re.IGNORECASE):
                                if "adnu" in headers:
                                    if "upgrade" not in headers:
                                        # Rename 'adnu' to 'upgrade'
                                        adnu_idx = headers.index("adnu")
                                        headers[adnu_idx] = "upgrade"
                                        for row in data_rows:
                                            row[adnu_idx] = " ".join([str(cell) for cell in row[adnu_idx] if pd.notna(cell)])
                                        print("Renamed 'adnu' to 'upgrade' in continuation ADNU table.", file=log_file)
                                # Ensure 'type of upgrade' column is filled
                                if "type of upgrade" not in headers:
                                    headers.append("type of upgrade")
                                    expected_columns.append("type of upgrade")
                                    for row in data_rows:
                                        row.append(specific_phrase)
                                    print("Added 'type of upgrade' column with specific phrase for continuation ADNU table.", file=log_file)

                        else:
                            # No header row detected, treat all rows as data points
                            print(f"No header row detected in continuation table on page {page_number + 1}, table {table_index + 1}. Treating all rows as data.", file=log_file)

                        # Create DataFrame for continuation table
                        if is_header_row:
                            try:
                                df_continuation = pd.DataFrame(data_rows, columns=headers)
                            except ValueError as ve:
                                print(f"Error creating DataFrame for continuation table on page {page_number + 1}, table {table_index + 1}: {ve}", file=log_file)
                                continue
                        else:
                            # Create DataFrame with expected_columns
                            # Handle cases where continuation table has more columns
                            standardized_data = []
                            for row in data_rows:
                                if len(row) < len(expected_columns):
                                    # Insert 'type of upgrade' or 'upgrade' with specific_phrase
                                    if re.search(r"Area\s*Delivery\s*Network\s*Upgrade", specific_phrase, re.IGNORECASE):
                                        # For ADNU tables, assume missing "upgrade" column
                                        missing_cols = len(expected_columns) - len(row)
                                        #row += [specific_phrase] * missing_cols
                                        data_rows = [row[:3] + [specific_phrase] + row[3:] for row in data_rows]
                                        print(f"Inserted '{specific_phrase}' for missing columns in ADNU continuation table on page {page_number + 1}, table {table_index + 1}.", file=log_file)
                                    else:
                                        # For non-ADNU tables, assume missing "type of upgrade" column
                                        missing_cols = len(expected_columns) - len(row)
                                        #row += [specific_phrase] * missing_cols
                                        data_rows = [ [specific_phrase]  for row in data_rows]
                                        print(f"Inserted '{specific_phrase}' for missing columns in non-ADNU continuation table on page {page_number + 1}, table {table_index + 1}.", file=log_file)
                                elif len(row) > len(expected_columns):
                                    # Add new columns with default names
                                    extra_cols = len(row) - len(expected_columns)
                                    for i in range(extra_cols):
                                        new_col_name = f"column{len(expected_columns) + 1 + i}"
                                        expected_columns.append(new_col_name)
                                        last_table[new_col_name] = pd.NA
                                        print(f"Added new column '{new_col_name}' for extra data in continuation table on page {page_number + 1}, table {table_index + 1}.", file=log_file)
                                    row = row[:len(expected_columns)]

                                row_dict = dict(zip(expected_columns, [clean_string_cell(cell) for cell in row]))

                                # Handle 'type of upgrade' column
                                if "type of upgrade" in row_dict and (pd.isna(row_dict["type of upgrade"]) or row_dict["type of upgrade"] == ""):
                                    row_dict["type of upgrade"] = specific_phrase
                                    print(f"Replaced None in 'type of upgrade' for a row in continuation table on page {page_number + 1}, table {table_index + 1}.", file=log_file)

                                standardized_data.append(row_dict)

                            try:
                                df_continuation = pd.DataFrame(standardized_data, columns=expected_columns)
                            except ValueError as ve:
                                print(f"Error creating DataFrame for continuation table on page {page_number + 1}, table {table_index + 1}: {ve}", file=log_file)
                                continue


                             # Special Handling for "Area Delivery Network Upgrade" Tables in Continuation
                            if re.search(r"Area\s*Delivery\s*Network\s*Upgrade", specific_phrase, re.IGNORECASE):
                                if "type of upgrade" in df_continuation.columns:
                                    first_row = df_continuation.iloc[0]
                                    if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                        print(f"Replacing 'None' in 'type of upgrade' for the first data row of continuation on page {page_number + 1}, table {table_index + 1}",file=log_file)
                                        df_continuation.at[0, "type of upgrade"] = specific_phrase
                                else:
                                    # If "type of upgrade" column does not exist, add it
                                    df_continuation["type of upgrade"] = specific_phrase
                                    print(f"'type of upgrade' column added with value '{specific_phrase}' for continuation on page {page_number + 1}, table {table_index + 1}",file=log_file)
                            else:
                                # General Handling for other tables
                                if "type of upgrade" in df_continuation.columns:
                                    first_row = df_continuation.iloc[0]
                                    if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                        print(f"Replacing 'None' in 'Type of Upgrade' for the first data row of continuation on page {page_number + 1}, table {table_index + 1}",file=log_file)
                                        df_continuation.at[0, "type of upgrade"] = specific_phrase
                                else:
                                    # If "Type of Upgrade" column does not exist, add it
                                    df_continuation["type of upgrade"] = specific_phrase
                                    print(f"'Type of Upgrade' column added with value '{specific_phrase}' for continuation on page {page_number + 1}, table {table_index + 1}",file=log_file)


                        # Handle ADNU-specific logic in continuation tables
                        #if re.search(r"Area\s*Delivery\s*Network\s*Upgrade", specific_phrase, re.IGNORECASE):
                        #    print("Handling ADNU-specific logic in continuation table.", file=log_file)
                        #    if "upgrade" in df_continuation.columns and "adnu" not in df_continuation.columns:
                        #        # Ensure 'upgrade' column is present
                        #        if "upgrade" not in df_continuation.columns:
                        #            df_continuation["upgrade"] = specific_phrase
                        #            print("Added 'upgrade' column to continuation ADNU table.", file=log_file)

                        # Ensure no duplicate columns
                        if df_continuation.columns.duplicated().any():
                            print(f"Duplicate columns detected in continuation table on page {page_number + 1}, table {table_index + 1}. Dropping duplicates.", file=log_file)
                            df_continuation = df_continuation.loc[:, ~df_continuation.columns.duplicated()]

                        # Merge with the last extracted table
                        extracted_tables[-1] = pd.concat([extracted_tables[-1], df_continuation], ignore_index=True, sort=False)
                        print(f"Appended continuation table data to the last extracted table on page {page_number + 1}, table {table_index + 1}.", file=log_file)

    except Exception as e:
        print(f"Error processing Table 3 in {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
        return pd.DataFrame()

    # After processing all tables, concatenate them
    if extracted_tables:
        all_columns = set()
        for df in extracted_tables:
            all_columns.update(df.columns.tolist())

        standardized_tables = []
        for df in extracted_tables:
            standardized_df = df.reindex(columns=all_columns)
            standardized_tables.append(standardized_df)

        print("\nConcatenating all extracted Table 3 data...", file=log_file)
        try:
            table7_data = pd.concat(standardized_tables, ignore_index=True, sort=False)
            print(f"Successfully concatenated {len(standardized_tables)} tables.", file=log_file)
        except Exception as e:
            print(f"Error concatenating tables: {e}", file=log_file)
            print(traceback.format_exc(), file=log_file)
            table7_data = pd.DataFrame()
    else:
        print("No Table 3 data extracted.", file=log_file)
        table7_data = pd.DataFrame()

    return table7_data


'''
def extract_table7_and_replace_none(pdf_path, project_id, log_file, is_addendum=False):
    """Extracts Table 3 data and merges with base data."""
    base_data = extract_base_data(pdf_path, project_id, log_file)
    table7_data = extract_table7(pdf_path, log_file, is_addendum)

    if table7_data.empty:
        return base_data
    else:
        # Identify overlapping columns excluding 'point_of_interconnection'
        overlapping_columns = base_data.columns.intersection(table7_data.columns).difference(['point_of_interconnection'])
        table7_data = table7_data.drop(columns=overlapping_columns, errors='ignore')
        
        # Repeat base data for each row in table7_data
        base_data_repeated = pd.concat([base_data] * len(table7_data), ignore_index=True)
        
        try:
            # Concatenate base data with Table 3 data along columns
            merged_df = pd.concat([base_data_repeated, table7_data], axis=1, sort=False)
            
            # Ensure 'point_of_interconnection' is present and correctly populated
            if 'point_of_interconnection' not in merged_df.columns:
                merged_df['point_of_interconnection'] = base_data['point_of_interconnection'].iloc[0]
                print(f"Added 'point_of_interconnection' to merged data for {pdf_path}.", file=log_file)
            
            print(f"Merged base data with Table 3 data for {pdf_path}.", file=log_file)
            return merged_df
        except Exception as e:
            print(f"Error merging base data with Table 3 data for {pdf_path}: {e}", file=log_file)
            print(traceback.format_exc(), file=log_file)
            return base_data  # Fallback to base data only
'''

def extract_table7_and_replace_none(pdf_path, project_id, log_file, is_addendum=False):
    """
    Extracts Table 3 data and merges with base data.
    Returns:
      df       – either base_data or base_data×Table4 rows merged
      status   – one of "no_marker", "failed", or "success"
    """
    # 1) Pull out base data
    base_data   = extract_base_data(pdf_path, project_id, log_file)
    # 2) Did we even see a Table 3 marker in the text?
    has_marker  = check_has_table7(pdf_path)
    if not has_marker:
        print(f"No Table 3 marker found in {os.path.basename(pdf_path)}; skipping extraction.", 
              file=log_file)
        return base_data, "no_marker"

    # 3) Try to scrape Table 3
    table7_data = extract_table7(pdf_path, log_file, is_addendum)
    if table7_data.empty:
        print(f"Table 3 marker found in {os.path.basename(pdf_path)}, "
              f"but extraction returned empty DataFrame.", file=log_file)
        return base_data, "failed"

    # 3) We got actual rows → merge and return
    #    Drop any overlapping columns first
    overlapping = base_data.columns.intersection(table7_data.columns)
    if not overlapping.empty:
        table7_data = table7_data.drop(columns=overlapping, errors="ignore")

    #    Repeat base_data for each row of table7_data
    base_rep   = pd.concat([base_data] * len(table7_data), ignore_index=True)
    merged_df  = pd.concat([base_rep, table7_data.reset_index(drop=True)], axis=1, sort=False)

    print(f"Merged base data with {len(table7_data)} row(s) of Table 3 for "
          f"{os.path.basename(pdf_path)}.", file=log_file)
    return merged_df, "success"





def check_has_table7(pdf_path):
    """Checks if the PDF contains Table 3-1 to 3-3."""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text = page.extract_text() or ""
                if re.search(r"Table\s*[C-D][\.-]1\b", text, re.IGNORECASE):
                    return True
    except Exception as e:
        # Handle potential errors when opening PDF
        return False
    return False

def has_network_upgrade_type_column(pdf_path, log_file):
    """Checks if any table in the PDF has a column header 'Network Upgrade Type'."""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page_number, page in enumerate(pdf.pages, start=1):
                tables = page.find_tables()
                for table_index, table in enumerate(tables, start=1):
                    tab = table.extract()
                    if not tab:
                        continue
                    headers = clean_column_headers(tab[0])
                    if "network upgrade type" in headers:
                        print(f"Found 'Network Upgrade Type' in PDF {pdf_path} on page {page_number}, table {table_index}.", file=log_file)
                        return True
    except Exception as e:
        print(f"Error checking 'Network Upgrade Type' in {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
    return False

def is_addendum(pdf_path, log_file):
    """Checks if the PDF is an addendum by searching 'Addendum' or 'Revision' on the first page."""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            if len(pdf.pages) == 0:
                return False
            first_page = pdf.pages[0]
            text = first_page.extract_text() or ""
            print(f"Extracted Text: {text}", file= log_file)  # Debugging step
            # Case-insensitive check for 'Addendum' or 'Revision'
            text_lower = text.lower()
            return "addendum" in text_lower or "revision" in text_lower
    except Exception as e:
        # Handle potential errors when opening PDF
        return False

def make_unique_headers(headers):
    """
    Appends a suffix to duplicate headers to make them unique.

    Args:
        headers (list): List of column headers.

    Returns:
        list: List of unique column headers.
    """
    seen = {}
    unique_headers = []
    for header in headers:
        if header in seen:
            seen[header] += 1
            unique_headers.append(f"{header}_{seen[header]}")
        else:
            seen[header] = 1
            unique_headers.append(header)
    return unique_headers

def process_pdfs_in_folder():
    """Processes all PDFs in the specified project range and directory."""
    global core_originals, core_addendums, total_pdfs_accessed, total_pdfs_scraped, total_pdfs_skipped, total_pdfs_skipped_extraction

    SKIP_PROJECTS = {1860, 2003, 2006}

    # Ensure the log file directory exists
    os.makedirs(os.path.dirname(LOG_FILE_PATH), exist_ok=True)

    with open(LOG_FILE_PATH, 'w') as log_file:


        for project_id in projects_to_process:
            
            # Skip the projects in the SKIP_PROJECTS set
            if project_id in SKIP_PROJECTS:
                print(f"Skipping Project {project_id} (marked to skip)", file=log_file)
                continue

         
            project_path = os.path.join(BASE_DIRECTORY, str(project_id), "03_phase_2_study")
            if not os.path.exists(project_path):
                missing_projects.add(project_id)
                print(f"Project path does not exist: {project_path}", file=log_file)
                continue

            project_scraped = False  # Flag to track if any PDF in the project was scraped
            base_data_extracted = False
            base_data = pd.DataFrame()

            # **START OF CHANGES**
            # Separate PDFs into originals and addendums
            list_pdfs = [pdf for pdf in os.listdir(project_path) if pdf.endswith(".pdf")]
            originals = []
            addendums = []
            for pdf_name in list_pdfs:
                pdf_path = os.path.join(project_path, pdf_name)
                if is_addendum(pdf_path, log_file):
                    addendums.append(pdf_name)
                else:
                    originals.append(pdf_name)
            # **END OF CHANGES**

            # **START OF CHANGES**
            # Process original PDFs first
            for pdf_name in originals:
                
                pdf_path = os.path.join(project_path, pdf_name)
                total_pdfs_accessed += 1

                is_add = is_addendum(pdf_path, log_file)

                # Check if PDF has 'Network Upgrade Type' column
                if has_network_upgrade_type_column(pdf_path, log_file):
                    style_n_pdfs.append(pdf_name)
                    print(f"Skipping PDF: {pdf_name} from Project {project_id} (Style N)", file=log_file)
                    # Still check if original has table7
                    has_table7 = check_has_table7(pdf_path)
                    original_has_table7[project_id] = has_table7
                    continue  # Skip processing this PDF

                print(f"Accessing Original PDF: {pdf_name} from Project {project_id}", file=log_file)
                original_pdfs.append(pdf_name)

                try:
                    has_table7 = check_has_table7(pdf_path)
                    original_has_table7[project_id] = has_table7

                    if not has_table7:
                        skipped_pdfs.append(pdf_name)
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} (No Table 3)", file=log_file)
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} (No Table 3)")
                        total_pdfs_skipped += 1
                        continue

                    if not base_data_extracted:
                        # Extract base data from original PDF
                        base_data = extract_base_data(pdf_path, project_id, log_file)
                        base_data_extracted = True
                        print(f"Extracted base data from original PDF: {pdf_name}", file=log_file)

                    # Extract Table 3 and merge
                    '''
                    df = extract_table7_and_replace_none(pdf_path, project_id, log_file, is_addendum=False)
                    if not df.empty:
                        core_originals = pd.concat([core_originals, df], ignore_index=True)
                        scraped_pdfs.append(pdf_name)
                        scraped_projects.add(project_id)
                        project_scraped = True
                        total_pdfs_scraped += 1
                        print(f"Scraped PDF: {pdf_name} from Project {project_id}")
                    else:
                        skipped_pdfs.append(pdf_name)
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} (Empty Data)", file=log_file)
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} (Empty Data)")
                        total_pdfs_skipped += 1
                    '''
                        # Extract Table 3 and merge
                    df, status = extract_table7_and_replace_none(pdf_path, project_id, log_file, is_addendum=False)

                    if status == "success":
                        core_originals = pd.concat([core_originals, df], ignore_index=True)
                        scraped_pdfs.append(pdf_name)
                        scraped_projects.add(project_id)
                        project_scraped = True
                        total_pdfs_scraped += 1
                        print(f"Scraped PDF: {pdf_name} from Project {project_id}")

                    elif status == "failed":
                        skipped_pdfs.append(pdf_name)
                        total_pdfs_skipped_extraction += 1
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} (Table 3 found but extraction failed)"
                             )

                    else:  # status == "no_marker"
                        skipped_pdfs.append(pdf_name)
                        total_pdfs_skipped += 1
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} (No Table 3 present)" )


                except Exception as e:
                    skipped_pdfs.append(pdf_name)
                    print(f"Skipped PDF: {pdf_name} from Project {project_id} due to an error: {e}", file=log_file)
                    print(traceback.format_exc(), file=log_file)
                    print(f"Skipped PDF: {pdf_name} from Project {project_id} due to an error: {e}")
                    total_pdfs_skipped += 1
            # **END OF CHANGES**

            # **START OF CHANGES**
            # Then process addendum PDFs
            for pdf_name in addendums:
                pdf_path = os.path.join(project_path, pdf_name)
                total_pdfs_accessed += 1
                is_add = is_addendum(pdf_path, log_file)

                # Check if PDF has 'Network Upgrade Type' column
                if has_network_upgrade_type_column(pdf_path, log_file):
                    style_n_pdfs.append(pdf_name)
                    print(f"Skipping PDF: {pdf_name} from Project {project_id} (Style N)", file=log_file)
                    continue  # Skip processing this PDF

                print(f"Accessing Addendum PDF: {pdf_name} from Project {project_id}", file=log_file)
                addendum_pdfs.append(pdf_name)

                try:
                    has_table7 = check_has_table7(pdf_path)

                    if not has_table7:
                        if original_has_table7.get(project_id, False):
                            # Attempt to scrape alternative tables is no longer needed
                            # According to the latest request, alternative table scraping is removed
                            # Therefore, we skip addendum PDFs that do not have Table 3
                            skipped_pdfs.append(pdf_name)
                            print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (No Table 3)", file=log_file)
                            print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (No Table 3)")
                            total_pdfs_skipped += 1
                        else:
                            skipped_pdfs.append(pdf_name)
                            print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (No Table 3 and original does not have Table 3)", file=log_file)
                            print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (No Table 3 and original does not have Table 3)")
                            total_pdfs_skipped += 1
                        continue

                    if not is_add and not base_data_extracted:
                        # Extract base data from original PDF
                        base_data = extract_base_data(pdf_path, project_id, log_file)
                        base_data_extracted = True
                        print(f"Extracted base data from original PDF: {pdf_name}", file=log_file)

                    if is_add and base_data_extracted:
                        # For addendums, use the extracted base data
                        table7_data = extract_table7(pdf_path, log_file, is_addendum=is_add)
                        if table7_data.empty and original_has_table7.get(project_id, False):
                            # Scrape alternative tables is removed, so skip if no data
                            skipped_pdfs.append(pdf_name)
                            print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (No relevant tables found)", file=log_file)
                            print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (No relevant tables found)")
                            total_pdfs_skipped += 1
                        if not table7_data.empty:
                            # Merge base data with Table 3 data
                            merged_df = pd.concat([base_data] * len(table7_data), ignore_index=True)
                            merged_df = pd.concat([merged_df, table7_data], axis=1, sort=False)
                            core_addendums = pd.concat([core_addendums, merged_df], ignore_index=True)
                            scraped_pdfs.append(pdf_name)
                            scraped_projects.add(project_id)
                            project_scraped = True
                            total_pdfs_scraped += 1
                            print(f"Scraped Addendum PDF: {pdf_name} from Project {project_id}")
                        else:
                            skipped_pdfs.append(pdf_name)
                            print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (Empty Data)", file=log_file)
                            print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (Empty Data)")
                            total_pdfs_skipped += 1
                except Exception as e:
                    skipped_pdfs.append(pdf_name)
                    print(f"Skipped PDF: {pdf_name} from Project {project_id} due to an error: {e}", file=log_file)
                    print(traceback.format_exc(), file=log_file)
                    # Optionally, print to ipynb
                    print(f"Skipped PDF: {pdf_name} from Project {project_id} due to an error: {e}")
                    total_pdfs_skipped += 1
            # **END OF CHANGES**

            # After processing all PDFs for this project, check if any PDF was scraped
            if not project_scraped and os.path.exists(project_path):
                skipped_projects.add(project_id)

    # Rest of the code remains unchanged...

    # After processing all PDFs, save to CSV
    save_to_csv(core_originals, OUTPUT_CSV_PATH_ORIGINAL, "originals")
    save_to_csv(core_addendums, OUTPUT_CSV_PATH_ADDENDUM, "addendums")

    # Calculate total projects processed
    total_projects_processed = len(scraped_projects) + len(skipped_projects)

    # Print summary to ipynb
    print("\n=== Scraping Summary ===")
    print(f"Total Projects Processed: {total_projects_processed}")
    print(f"Total Projects Scraped: {len(scraped_projects)}")
    print(f"Total Projects Skipped due to failed extraction of Table: {total_pdfs_skipped_extraction}")
    print(f"Total Projects Skipped: {len(skipped_projects)}")
    print(f"Total Projects Missing: {len(missing_projects)}")
    print(f"Total PDFs Accessed: {total_pdfs_accessed}")
    print(f"Total PDFs Scraped: {total_pdfs_scraped}")
    print(f"Total PDFs Skipped: {total_pdfs_skipped}")

    print("\nList of Scraped Projects:")
    print(sorted(scraped_projects))

    print("\nList of Skipped Projects:")
    print(sorted(skipped_projects))

    print("\nList of Missing Projects:")
    print(sorted(missing_projects))

    print("\nList of Scraped PDFs:")
    print(scraped_pdfs)

    print("\nList of Skipped PDFs:")
    print(skipped_pdfs)

    print("\nList of Addendum PDFs:")
    print(addendum_pdfs)

    print("\nList of Original PDFs:")
    print(original_pdfs)

    print("\nList of Style N PDFs (Skipped due to 'Network Upgrade Type'):")
    print(style_n_pdfs)

    print("\nTotal Number of Style N PDFs:", len(style_n_pdfs))

    print("\nNumber of Original PDFs Scraped:", len([pdf for pdf in scraped_pdfs if pdf in original_pdfs]))
    print("Number of Addendum PDFs Scraped:", len([pdf for pdf in scraped_pdfs if pdf in addendum_pdfs]))

def save_to_csv(df, output_csv_path, data_type):
    """Cleans the DataFrame and saves it to a CSV file."""
    if df.empty:
        print(f"No data to save for {data_type}.")
        return

    # Clean up the entire DataFrame by cleaning string cells
    df = df.applymap(clean_string_cell)

    # Drop rows that contain specific phrases (e.g., "Type of Upgrade")
    df = df[~df.apply(lambda row: contains_phrase(row, "Type of Upgrade"), axis=1)]

    # Reorder columns as specified
    df = reorder_columns(df)
    print(f"\nColumns reordered for {data_type} as per specification.")

    # Ensure q_id is numeric for sorting, replace missing values with None
 

    # Save the DataFrame to CSV
    try:
        df.to_csv(output_csv_path, index=False)
        print(f"\nData successfully saved to {output_csv_path}")
    except Exception as e:
        print(f"Error saving {data_type} data to CSV: {e}")
        print(traceback.format_exc())

def main():
    """Main function to execute the PDF scraping process."""
    process_pdfs_in_folder()

if __name__ == "__main__":
    main()


Will process these project‐folders:
['643', '643AS', '643AA', '643AF', '643AH', '643AI', '643I', '643G', '643R', '643F', '643O', '643Z', '643S', '643T', '643AE', '643AB', '643AK', '643AP', '643AM', '643AJ', '643AC', '643X', '643J', '643D', '643W', '643E']
Skipped PDF: C3C4P2-Northern-Q643AA-Blue Sky Wind Energy Center-AppendixA.pdf from Project 643AA (Table 3 found but extraction failed)
Skipped Addendum PDF: 10AS665623-QC34PIINorthernQ643AAAddendum_20121227.pdf from Project 643AA (No relevant tables found)
Skipped Addendum PDF: 10AS665623-QC34PIINorthernQ643AAAddendum_20121227.pdf from Project 643AA (Empty Data)
Skipped PDF: C3C4P2-SCE EOP-Q643AI-AppendixA.pdf from Project 643AI (Table 3 found but extraction failed)
Skipped Addendum PDF: 10AS670281-C3C4PII_Q643AI_Addendum.pdf from Project 643AI (No Table 3)


KeyboardInterrupt: 

# OCR

In [22]:
import os
import pdfplumber
import pandas as pd
import re
import PyPDF2
import traceback
import io
import pytesseract

# Define paths and project range
BASE_DIRECTORY ="/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data"
OUTPUT_CSV_PATH_ORIGINAL = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/03_raw/ph2_rawdata_cluster3_style_new_originals.csv"
OUTPUT_CSV_PATH_ADDENDUM = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/03_raw/ph2_rawdata_cluster3_style_new_addendums.csv"
LOG_FILE_PATH = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/03_raw/ph2_scraping_cluster3_style_new_log.txt"
PROJECT_RANGE = range(643, 644)  # Inclusive range for q_ids in Clusters 3 range(667, 860)


# Read the CSV file containing processed projects (with q_id column)
processed_csv_path = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/all_clusters/costs_phase_2_all_clusters_total.csv"  # UPDATE THIS PATH
processed_df = pd.read_csv(processed_csv_path)
# Convert q_id values to numeric then to int for filtering
processed_q_ids = pd.to_numeric(processed_df['q_id'], errors='coerce').dropna().astype(int).unique()
# Now build the list of folders to process:
projects_to_process = []
for folder in os.listdir(BASE_DIRECTORY):
    full = os.path.join(BASE_DIRECTORY, folder)
    if not os.path.isdir(full):
        continue
    m = re.match(r'^(\d+)', folder)
    if not m:
        continue
    num = int(m.group(1))
    if num in PROJECT_RANGE and num not in processed_q_ids:
        projects_to_process.append(folder)

print("Will process these project‐folders:")
print(projects_to_process)

# Initialize DataFrames
core_originals = pd.DataFrame()
core_addendums = pd.DataFrame()

# Initialize tracking variables
scraped_projects = set()
skipped_projects = set()
missing_projects = set()
scraped_pdfs = []
skipped_pdfs = []
addendum_pdfs = []
original_pdfs = []
style_n_pdfs = []  # List to track style N PDFs
total_pdfs_accessed = 0
total_pdfs_scraped = 0
total_pdfs_skipped = 0
total_pdfs_skipped_extraction = 0
original_has_table7 = {}  # Dictionary to track if original PDFs have table7

def clean_column_headers(headers):
    """Cleans column headers by normalizing and removing unwanted characters, but keeps parentheses."""
    cleaned_headers = []
    for header in headers:
        if header is None:
            header = ""
        elif isinstance(header, str):
            header = header.lower()
            # collapse internal whitespace
            header = re.sub(r'\s+', ' ', header)
            # strip out everything except letters, digits, spaces, and parentheses
            header = re.sub(r'[^a-z0-9\s\(\)]', '', header)
            header = header.strip()
        cleaned_headers.append(header)
    return cleaned_headers

def clean_string_cell(value):
    """Cleans string cells by removing newlines and trimming spaces."""
    if isinstance(value, str):
        return value.replace('\n', ' ').strip()
    elif value is None:
        return ""
    else:
        return str(value).replace('\n', ' ').strip()
     

def contains_phrase(row, phrase):
    """Checks if any cell in a row contains a specific phrase."""
    regex_pattern = re.sub(r"\s+", r"\\s*", phrase)
    pattern = re.compile(regex_pattern, flags=re.IGNORECASE)
    return row.astype(str).apply(lambda cell: bool(pattern.search(cell))).any()

def extract_specific_phrase(title):
    """
    Extracts a specific phrase from the table title based on predefined keywords.

    Args:
        title (str): The table title string.

    Returns:
        str: The extracted specific phrase if found, else the original title.
    """
    phrases = [
        "PTO",
        "Reliability Network Upgrade",
        "Area Delivery Network Upgrade",
        "Local Delivery Network",
         
        
        "Other Potential Network Upgrade",
        "Area Delivery Network Upgrades",
        "Conditionally Assigned Network Upgrades",
        "Local Off-Peak Network Upgrade",
        "ADNU",
        "LDNU",
        "RNU"
    ]

    for phrase in phrases:
        if  re.search(rf"\b{re.escape(phrase)}\b(?=\d|\W|$)", title, re.IGNORECASE):
        
         #re.search(rf"\b{re.escape(phrase)}\b", title, re.IGNORECASE):
            return phrase
    return title  # Fallback to the entire title if no specific phrase is found

def reorder_columns(df):
    """
    Reorders the columns of the DataFrame based on the specified order.

    Args:
        df (pd.DataFrame): The DataFrame to reorder.

    Returns:
        pd.DataFrame: The reordered DataFrame.
    """
    desired_order = [
        "q_id",
        "cluster",
        "req_deliverability",
        "latitude",
        "longitude",
        "capacity",
        "point_of_interconnection",
        "type of upgrade",
        "upgrade",
        "description",
        "cost allocation factor"
    ]

    # Start with desired columns that exist in the DataFrame
    existing_desired = [col for col in desired_order if col in df.columns]

    # Then add the remaining columns
    remaining = [col for col in df.columns if col not in existing_desired]

    # Combine the two lists
    new_order = existing_desired + remaining

    # Reorder the DataFrame
    df = df[new_order]

    return df

def search_gps_coordinates(text, log_file):
    """Search for GPS coordinates using multiple patterns."""
    gps_coords = re.search(r"gps coordinates:\s*([\d\.\-]+),\s*([\d\.\-]+)", text, re.IGNORECASE)
    if gps_coords:
        print(f"Found GPS coordinates: {gps_coords.groups()}", file=log_file)
        return gps_coords.groups()

    project_coords = re.search(r"latitude[:\s]*([\d\.\-]+)[^\d]+longitude[:\s]*([\d\.\-]+)", text, re.IGNORECASE)
    if project_coords:
        print(f"Found project coordinates: {project_coords.groups()}", file=log_file)
        return project_coords.groups()

    gps_coords_directional = re.search(
        r"gps coordinates:\s*([\d\.\-]+)\s*[nNsS],\s*([\d\.\-]+)\s*[eEwW]", text, re.IGNORECASE)
    if gps_coords_directional:
        lat, lon = gps_coords_directional.groups()
        latitude = lat if "N" in text.upper() else f"-{lat}"  # Adjust latitude sign
        longitude = lon if "E" in text.upper() else f"-{lon}"  # Adjust longitude sign
        print(f"Found directional GPS coordinates: {(latitude, longitude)}", file=log_file)
        return (latitude, longitude)

    print("GPS coordinates not found.", file=log_file)
    return (None, None)

def extract_table1(pdf_path, log_file):
    """
    Extracts the Point of Interconnection from Table 1 in the provided PDF.
    Implements a retry mechanism with different table extraction settings if initial attempts fail.

    Args:
        pdf_path (str): Path to the PDF file.
        log_file (file object): Log file to write print statements.

    Returns:
        str: Extracted Point of Interconnection value,
             "Value Missing" if label found but no value,
             or None if not found.
    """
    print(f"\nProcessing {pdf_path} for Table 1 extraction...", file=log_file)
    point_of_interconnection = None

    # Define the regex pattern for 'Point of Interconnection' (case-insensitive)
    poi_pattern = re.compile(r"Point\s+of\s+Interconnection", re.IGNORECASE)

    # Define different table extraction settings to try
    table_settings_list = [
        {
            "horizontal_strategy": "text",
            "vertical_strategy": "lines",
            "snap_tolerance": 1,
        },
        {
            "horizontal_strategy": "lines",
            "vertical_strategy": "lines",
            "snap_tolerance": 2,  # Increased tolerance for retry
        }
    ]

    try:
        with pdfplumber.open(pdf_path) as pdf:
            # Identify all pages that contain "Table 1"
            table1_pages = []
            for i, page in enumerate(pdf.pages):
                text = page.extract_text() or ""
                if re.search(r"Table\s*B.1\b", text, re.IGNORECASE):
                    table1_pages.append(i)

            if not table1_pages:
                print("No Table 1 found in the PDF.", file=log_file)
                return None  # Return None if no Table 1 found

            first_page = table1_pages[0]
            last_page = table1_pages[-1]
            scrape_start = first_page
            scrape_end = last_page + 2  # Plus one to include the next page if needed

            print(f"Table 1 starts on page {scrape_start + 1} and ends on page {scrape_end + 1}", file=log_file)

            # Flag to indicate if extraction was successful
            extraction_successful = False

            # Iterate through the specified page range
            for page_number in range(scrape_start, min(scrape_end + 1, len(pdf.pages))):
                page = pdf.pages[page_number]
                print(f"\nScraping tables on page {page_number + 1} for Table 1...", file=log_file)

                for attempt, table_settings in enumerate(table_settings_list, start=1):
                    print(f"\nAttempt {attempt} with table settings: {table_settings}", file=log_file)
                    tables = page.find_tables(table_settings=table_settings)
                    print(f"Found {len(tables)} table(s) on page {page_number + 1} with current settings.", file=log_file)

                    for table_index, table in enumerate(tables, start=1):
                        tab = table.extract()
                        if not tab:
                            print(f"Table {table_index} on page {page_number + 1} is empty. Skipping.", file=log_file)
                            continue  # Skip empty tables

                        print(f"\n--- Table {table_index} on Page {page_number + 1} ---", file=log_file)
                        for row_num, row in enumerate(tab, start=1):
                            print(f"Row {row_num}: {row}", file=log_file)

                        # Iterate through each row in the table
                        for row_index, row in enumerate(tab, start=1):
                            # Iterate through each cell in the row
                            for cell_index, cell in enumerate(row, start=1):
                                if cell and poi_pattern.search(cell):
                                    # Assuming the next column contains the value
                                    poi_col_index = cell_index  # 1-based index
                                    adjacent_col_index = poi_col_index + 1  # Next column

                                    if adjacent_col_index <= len(row):
                                        poi_value = clean_string_cell(row[adjacent_col_index - 1])
                                        if poi_value:  # Check if the value is not empty
                                            point_of_interconnection = poi_value
                                            print(f"\nFound Point of Interconnection: '{point_of_interconnection}' "
                                                  f"(Page {page_number + 1}, Table {table_index}, Row {row_index})", file=log_file)
                                            extraction_successful = True
                                            break  # Exit the cell loop
                                        else:
                                            print(f"\nPoint of Interconnection label found but adjacent value is empty "
                                                  f"(Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                                            # Proceed to scan surrounding rows for the value
                                            poi_value_parts = []

                                            # Define the range to scan: two rows above and two rows below
                                            # Convert to 0-based index
                                            current_row_idx = row_index - 1
                                            start_scan = max(0, current_row_idx - 2)
                                            end_scan = min(len(tab), current_row_idx + 3)  # Exclusive

                                            print(f"Scanning rows {start_scan + 1} to {end_scan} for POI value parts.", file=log_file)

                                            for scan_row_index in range(start_scan, end_scan):
                                                # Skip the current row where the label was found
                                                if scan_row_index == current_row_idx:
                                                    continue

                                                scan_row = tab[scan_row_index]
                                                # Ensure the adjacent column exists in the scan row
                                                if adjacent_col_index - 1 < len(scan_row):
                                                    scan_cell = clean_string_cell(scan_row[adjacent_col_index - 1])
                                                    if scan_cell and not poi_pattern.search(scan_cell):
                                                        poi_value_parts.append(scan_cell)
                                                        print(f"Found POI part in row {scan_row_index + 1}: '{scan_cell}'", file=log_file)
                                                    elif poi_pattern.search(scan_cell):
                                                        # If another POI label is found, skip it
                                                        print(f"Encountered another POI label in row {scan_row_index + 1}. Skipping this row.", file=log_file)
                                                        continue

                                            if poi_value_parts:
                                                # Concatenate the parts to form the complete POI value
                                                point_of_interconnection = " ".join(poi_value_parts)
                                                print(f"\nConcatenated Point of Interconnection: '{point_of_interconnection}' "
                                                      f"(Page {page_number + 1}, Table {table_index})", file=log_file)
                                                extraction_successful = True
                                                break  # Exit the cell loop
                                            else:
                                                print(f"\nNo POI value found in the surrounding rows "
                                                      f"(Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                                                # Do not return immediately; proceed to retry
                                    else:
                                        print(f"\nPoint of Interconnection label found but no adjacent column "
                                              f"(Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                                        # Do not return immediately; proceed to retry
                            if extraction_successful:
                                break  # Exit the row loop
                        if extraction_successful:
                            break  # Exit the table loop
                    if extraction_successful:
                        break  # Exit the attempt loop
                if extraction_successful:
                    break  # Exit the page loop

    except Exception as e:
        print(f"Error processing Table 1 in {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
        return None

    if not extraction_successful:
        # After all attempts, determine the appropriate return value
        if point_of_interconnection is not None and point_of_interconnection != "":
            # Label was found but no value
            print("Point of Interconnection label found but no adjacent value.", file=log_file)
            return "Value Missing"
        else:
            # Label not found
            print("Point of Interconnection not found in Table 1.", file=log_file)
            return None

    return point_of_interconnection

def extract_base_data(pdf_path, project_id, log_file):
    """Extract base data from the PDF and return as a DataFrame."""
    print("Extracting base data from PDF...", file=log_file)
    try:
        with open(pdf_path, 'rb') as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text

        text = clean_string_cell(text)

        queue_id = re.search(r"q[\s_-]*(\d+)", text, re.IGNORECASE)
        queue_id =  str(project_id)  # Use project_id if queue_id is not found
        print(f"Extracted Queue ID: {queue_id}", file=log_file)

        # Updated Cluster Extraction
        clusters = re.findall(r"queue[\s_-]*cluster[\s_-]*(\d+)", text, re.IGNORECASE)
        if '3' in clusters:
            cluster_number = '3'
        elif clusters:
            cluster_number = max(clusters, key=lambda x: int(x))  # Choose the highest cluster number found
        else:
            cluster_number = '3'  # Default to 3 if not found
        print(f"Extracted Cluster Number: {cluster_number}", file=log_file)

        deliverability_status = re.search(r"(\w+)\s*capacity deliverability status", text, re.IGNORECASE)
        deliverability_status = deliverability_status.group(1) if deliverability_status else None
        print(f"Extracted Deliverability Status: {deliverability_status}", file=log_file)

        # Extract Capacity
        capacity = re.search(r"total rated output of (\d+)\s*mw", text, re.IGNORECASE)
        if capacity:
            capacity = int(capacity.group(1))
        else:
            capacity2 = re.search(r"(\d+)\s*mw", text)
            capacity = int(capacity2.group(1)) if capacity2 else None
        print(f"Extracted Capacity: {capacity}", file=log_file)

        # Extract Point of Interconnection
        point_of_interconnection = extract_table1(pdf_path, log_file)

        latitude, longitude = search_gps_coordinates(text, log_file)

        # Initialize base data dictionary
        base_data = {
            "q_id": [queue_id],
            "cluster": [cluster_number],
            "req_deliverability": [deliverability_status],
            "latitude": [latitude],
            "longitude": [longitude],
            "capacity": [capacity],
            "point_of_interconnection": [point_of_interconnection]
        }

        print("Base data extracted:", file=log_file)
        print(base_data, file=log_file)
        return pd.DataFrame(base_data)

    except Exception as e:
        print(f"Error extracting base data from {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
        return pd.DataFrame()  # Return empty DataFrame on error

def adjust_rows_length(data_rows, headers):
    """Ensure each row in data_rows matches the length of headers by truncating or padding."""
    col_count = len(headers)
    for i in range(len(data_rows)):
        row = data_rows[i]
        if len(row) > col_count:
            data_rows[i] = row[:col_count]
        elif len(row) < col_count:
            data_rows[i].extend([""]*(col_count - len(row)))


def extract_table7(pdf_path, log_file, is_addendum=False):
    """
    Extracts Table 3 data from the provided PDF. Pages in the scrape range
    are first OCR’d (via pytesseract.image_to_pdf_or_hocr) and re-assembled
    into a temporary PDF in memory before any table extraction is attempted.

    Args:
        pdf_path (str): Path to the PDF file.
        log_file (file object): Log file to write debug/print statements.
        is_addendum (bool): Whether the PDF is an addendum (unused here, but kept for signature compatibility).

    Returns:
        pd.DataFrame: Concatenated DataFrame of all Table 3 data, or an empty DataFrame if none found.
    """
    print(f"\nProcessing {pdf_path} for Table 3 extraction...", file=log_file)

    try:
        # First pass: find which pages have a "Table C-1" / "Table D-1" marker in their text
        with pdfplumber.open(pdf_path) as pdf_orig:
            table7_pages = []
            for i, page in enumerate(pdf_orig.pages):
                raw = page.extract_text() or ""
                if re.search(r"Table\s*[C-D]\s*[\.-]\s*1\s*:", raw, re.IGNORECASE):
                    table7_pages.append(i)

            if not table7_pages:
                print("No Table 3-1 to 3-3 found in the PDF.", file=log_file)
                return pd.DataFrame()

            first_page = table7_pages[0]
            last_page = table7_pages[-1]
            scrape_start = first_page
            scrape_end = last_page + 3  # include two pages of possible continuation

            print(f"Table 3 starts on page {scrape_start} and ends on page {scrape_end}", file=log_file)

            # ---------- OCR step: build a temporary in-memory PDF where pages in [scrape_start:scrape_end]
            # are replaced by Tesseract OCR output. Others are copied verbatim. ----------
            writer = PyPDF2.PdfWriter()

            for idx, original_page in enumerate(pdf_orig.pages):
                if scrape_start <= idx < scrape_end and idx < len(pdf_orig.pages):
                    # Render page to a high-res PIL image, then ask Tesseract to return a one-page PDF
                    img = original_page.to_image(resolution=300).original
                    try:
                        ocr_pdf_bytes = pytesseract.image_to_pdf_or_hocr(img, extension='pdf')
                    except Exception as e:
                        print(f"  [OCR ERROR] on page {idx+1}: {e}", file=log_file)
                        # fallback: just copy the original vector page
                        writer.add_page(PyPDF2._page.PageObject.create_blank_page(
                            width=original_page.width, height=original_page.height
                        ))
                        continue

                    # Read the single-page PDF bytes back into PyPDF2
                    ocr_reader = PyPDF2.PdfReader(io.BytesIO(ocr_pdf_bytes))
                    if len(ocr_reader.pages) >= 1:
                        writer.add_page(ocr_reader.pages[0])
                        print(f"  [OCR] replaced page {idx+1} with OCR’d PDF page", file=log_file)
                    else:
                        # if something went wrong, just copy original
                        writer.add_page(pdf_orig.pages[idx].page_obj)
                else:
                    # copy original page object verbatim
                    writer.add_page(pdf_orig.pages[idx].page_obj)

            # Now write the in-memory PDF to a bytes buffer
            tmp_buffer = io.BytesIO()
            writer.write(tmp_buffer)
            tmp_buffer.seek(0)

        # Open the newly created OCR-augmented PDF via pdfplumber
        with pdfplumber.open(tmp_buffer) as pdf:
            extracted_tables = []
            specific_phrase = None

            # Now exactly the same “find tables” loops, but against our OCR’d version:
            for page_number in range(scrape_start, min(scrape_end, len(pdf.pages))):
                page = pdf.pages[page_number]
                print(f"\nScraping tables on page {page_number + 1}...", file=log_file)

                tables = page.find_tables(table_settings={
                    "horizontal_strategy": "lines",
                    "vertical_strategy": "lines",
                })

                for table_index, table in enumerate(tables):
                    tab = table.extract()
                    if not tab:
                        print(f"Table {table_index + 1} on page {page_number + 1} is empty. Skipping.", file=log_file)
                        continue

                    table_bbox = table.bbox
                    title_bbox = (0, 0, page.width, table_bbox[1])
                    title_text = page.within_bbox(title_bbox).extract_text() or ""
                    table_title = None

                    if title_text:
                        title_lines = title_text.split("\n")[::-1]
                        for line in title_lines:
                            line = line.strip()
                            match = re.search(r"(Modification\s+of\s+)?Table\s*[C-D][\.-]([1-2])[:\-\s]*(.*)", line, re.IGNORECASE)
                            if match:
                                try:
                                    table_title = match.group(3).strip()
                                except IndexError:
                                    table_title = match.group(0).strip()
                                    print("Fallback to whole match for table title", file=log_file)

                    if table_title:
                        if re.search(r"PTO Interconnection Facilities Cost Estimate Summary", table_title, re.IGNORECASE):
                            print(f"Skipping Table 8-1 PTO on page {page_number + 1}, table {table_index + 1}.", file=log_file)
                            continue
                        if re.search(r"\b8-7\b", table_title, re.IGNORECASE):
                            print(f"Skipping Table 3-3 on page {page_number + 1}, table {table_index + 1}.", file=log_file)
                            continue

                        # “New Table 3” detected
                        specific_phrase = extract_specific_phrase(table_title)
                        print(f"New Table 3 detected: '{specific_phrase}' on page {page_number + 1}, table {table_index + 1}", file=log_file)

                        headers = clean_column_headers(tab[0])
                        data_rows = tab[1:]

                        try:
                            df_new = pd.DataFrame(data_rows, columns=headers)
                        except ValueError as ve:
                            print(f"Error creating DataFrame for new table on page {page_number + 1}, table {table_index + 1}: {ve}", file=log_file)
                            continue

                        if df_new.empty:
                            # header-only stub; append empty DF with right columns
                            extracted_tables.append(pd.DataFrame(columns=headers))
                            print(f"Header-only Table 3 (‘{specific_phrase}’) detected on page {page_number+1}; waiting for continuation…", file=log_file)
                            continue

                        # ADNU grouping logic (unchanged)
                        if re.search(r"Area\s*Delivery\s*Network\s*Upgrade", specific_phrase, re.IGNORECASE):
                            print("Detected 'Area Delivery Network Upgrade' table (new).", file=log_file)
                            if "adnu" in df_new.columns:
                                if "type of upgrade" not in df_new.columns:
                                    adnu_values = df_new["adnu"].dropna().astype(str).tolist()
                                    grouped_adnu = " ".join(adnu_values)
                                    other_columns = df_new.drop(columns=["adnu"]).iloc[0].to_dict()

                                    df_grouped = pd.DataFrame({
                                        "upgrade": [grouped_adnu],
                                        "type of upgrade": [specific_phrase]
                                    })
                                    for col, value in other_columns.items():
                                        df_grouped[col] = value

                                    print("Grouped all 'adnu' rows into a single 'upgrade' row for new ADNU table.", file=log_file)
                                    df_new = df_grouped
                                else:
                                    if "upgrade" in df_new.columns:
                                        df_new.drop(columns=["adnu"], inplace=True)
                                        print("Dropped 'adnu' column to avoid duplicate 'upgrade'.", file=log_file)
                                    else:
                                        df_new.rename(columns={"adnu": "upgrade"}, inplace=True)
                                        print("Renamed 'adnu' to 'upgrade' in new ADNU table.", file=log_file)

                            if "type of upgrade" not in df_new.columns:
                                df_new["type of upgrade"] = specific_phrase
                                print("Added 'type of upgrade' to all rows in new ADNU table.", file=log_file)
                            else:
                                first_row = df_new.iloc[0]
                                if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                    df_new.at[0, "type of upgrade"] = specific_phrase
                                    print("Replaced None in 'type of upgrade' first row for new ADNU table.", file=log_file)
                        else:
                            if "type of upgrade" not in df_new.columns:
                                df_new["type of upgrade"] = specific_phrase
                                print("Added 'type of upgrade' to all rows in new non-ADNU table.", file=log_file)
                            else:
                                first_row = df_new.iloc[0]
                                if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                    df_new.at[0, "type of upgrade"] = specific_phrase
                                    print("Replaced None in 'type of upgrade' first row for new non-ADNU table.", file=log_file)

                        # Drop duplicate columns if any
                        if df_new.columns.duplicated().any():
                            df_new = df_new.loc[:, ~df_new.columns.duplicated()]
                            print("Dropped duplicate columns in new table.", file=log_file)

                        extracted_tables.append(df_new)

                    else:
                        # Continuation block
                        if not extracted_tables:
                            print(f"No previous Table 3 to continue with on page {page_number + 1}, table {table_index + 1}. Skipping.", file=log_file)
                            continue

                        last_table = extracted_tables[-1]
                        expected_columns = last_table.columns.tolist()

                        print(f"Continuation Table detected on page {page_number + 1}, table {table_index + 1}", file=log_file)
                        data_rows = tab

                        # Check if first row is header
                        header_keywords = ["type of upgrade", "adnu", "MW at POI"]
                        first_row = data_rows[0] if data_rows else []
                        is_header_row = any(
                            any(re.search(rf"\b{kw}\b", clean_string_cell(cell).lower()) for kw in header_keywords)
                            for cell in first_row
                        )

                        if is_header_row:
                            headers = clean_column_headers(first_row)
                            data_rows = data_rows[1:]
                            new_columns = [col for col in headers if col not in expected_columns]
                            if new_columns:
                                expected_columns.extend(new_columns)
                                for new_col in new_columns:
                                    last_table[new_col] = pd.NA
                                last_table = last_table.reindex(columns=expected_columns)
                                extracted_tables[-1] = last_table
                                print(f"Added new columns from continuation: {new_columns}", file=log_file)

                            # Fill missing “type of upgrade” if needed
                            if "type of upgrade" in headers:
                                idx_upgrade = headers.index("type of upgrade")
                                if pd.isna(data_rows[0][idx_upgrade]) or data_rows[0][idx_upgrade] == "":
                                    data_rows[0][idx_upgrade] = specific_phrase
                                    print(f"Replaced None in 'type of upgrade' in continuation header row.", file=log_file)
                            elif "upgrade" in headers:
                                idx_up = headers.index("upgrade")
                                if pd.isna(data_rows[0][idx_up]) or data_rows[0][idx_up] == "":
                                    data_rows[0][idx_up] = specific_phrase
                                    print(f"Replaced None in 'upgrade' in continuation header row.", file=log_file)
                            else:
                                headers.append("type of upgrade")
                                expected_columns.append("type of upgrade")
                                for r in data_rows:
                                    r.append(specific_phrase)
                                print(f"Inserted 'type of upgrade' column into continuation rows.", file=log_file)

                        else:
                            print(f"No header row detected in continuation; treating all rows as data.", file=log_file)

                        # Build a standardized row dict list
                        standardized_data = []
                        for row in data_rows:
                            # If too few columns, insert “type of upgrade” at correct position
                            if len(row) < len(expected_columns):
                                if re.search(r"Area\s*Delivery\s*Network\s*Upgrade", specific_phrase, re.IGNORECASE):
                                    data_rows = [r[:3] + [specific_phrase] + r[3:] for r in data_rows]
                                    print(f"Inserted '{specific_phrase}' for missing ADNU column in continuation.", file=log_file)
                                else:
                                    data_rows = [[specific_phrase] for r in data_rows]
                                    print(f"Inserted '{specific_phrase}' for missing 'type of upgrade' in continuation.", file=log_file)
                            elif len(row) > len(expected_columns):
                                extra_cols = len(row) - len(expected_columns)
                                for i in range(extra_cols):
                                    new_col_name = f"column{len(expected_columns)+1+i}"
                                    expected_columns.append(new_col_name)
                                    last_table[new_col_name] = pd.NA
                                    print(f"Added new column '{new_col_name}' for extra data in continuation.", file=log_file)
                                row = row[:len(expected_columns)]

                            row_dict = dict(zip(expected_columns, [clean_string_cell(cell) for cell in row]))
                            if "type of upgrade" in row_dict and (pd.isna(row_dict["type of upgrade"]) or row_dict["type of upgrade"] == ""):
                                row_dict["type of upgrade"] = specific_phrase
                                print(f"Filled missing 'type of upgrade' in continuation row.", file=log_file)
                            standardized_data.append(row_dict)

                        try:
                            df_cont = pd.DataFrame(standardized_data, columns=expected_columns)
                        except ValueError as ve:
                            print(f"Error creating DataFrame for continuation on page {page_number + 1}: {ve}", file=log_file)
                            continue

                        # ADNU special logic for continuation (unchanged)
                        if re.search(r"Area\s*Delivery\s*Network\s*Upgrade", specific_phrase, re.IGNORECASE):
                            if "type of upgrade" in df_cont.columns:
                                first_row = df_cont.iloc[0]
                                if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                    df_cont.at[0, "type of upgrade"] = specific_phrase
                                    print(f"Replaced None in 'type of upgrade' for first ADNU continuation row.", file=log_file)
                            else:
                                df_cont["type of upgrade"] = specific_phrase
                                print(f"Added 'type of upgrade' column with '{specific_phrase}' to ADNU continuation.", file=log_file)
                        else:
                            if "type of upgrade" in df_cont.columns:
                                first_row = df_cont.iloc[0]
                                if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                    df_cont.at[0, "type of upgrade"] = specific_phrase
                                    print(f"Replaced None in 'type of upgrade' for first continuation row.", file=log_file)
                            else:
                                df_cont["type of upgrade"] = specific_phrase
                                print(f"Added 'type of upgrade' column with '{specific_phrase}' to continuation.", file=log_file)

                        if df_cont.columns.duplicated().any():
                            df_cont = df_cont.loc[:, ~df_cont.columns.duplicated()]
                            print(f"Dropped duplicate columns in continuation DataFrame.", file=log_file)

                        extracted_tables[-1] = pd.concat([extracted_tables[-1], df_cont], ignore_index=True, sort=False)
                        print(f"Appended continuation table data to the last extracted table on page {page_number + 1}.", file=log_file)

            # After iterating all pages in [scrape_start:scrape_end], concatenate results
            if extracted_tables:
                all_columns = set()
                for df_part in extracted_tables:
                    all_columns.update(df_part.columns.tolist())

                standardized_tables = []
                for df_part in extracted_tables:
                    standardized_tables.append(df_part.reindex(columns=all_columns))

                print("\nConcatenating all extracted Table 3 data...", file=log_file)
                try:
                    table7_data = pd.concat(standardized_tables, ignore_index=True, sort=False)
                    print(f"Successfully concatenated {len(standardized_tables)} tables.", file=log_file)
                except Exception as e:
                    print(f"Error concatenating tables: {e}", file=log_file)
                    print(traceback.format_exc(), file=log_file)
                    table7_data = pd.DataFrame()
            else:
                print("No Table 3 data extracted.", file=log_file)
                table7_data = pd.DataFrame()

            return table7_data

    except Exception as e:
        print(f"Error processing Table 3 in {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
        return pd.DataFrame()
 


'''
def extract_table7_and_replace_none(pdf_path, project_id, log_file, is_addendum=False):
    """Extracts Table 3 data and merges with base data."""
    base_data = extract_base_data(pdf_path, project_id, log_file)
    table7_data = extract_table7(pdf_path, log_file, is_addendum)

    if table7_data.empty:
        return base_data
    else:
        # Identify overlapping columns excluding 'point_of_interconnection'
        overlapping_columns = base_data.columns.intersection(table7_data.columns).difference(['point_of_interconnection'])
        table7_data = table7_data.drop(columns=overlapping_columns, errors='ignore')
        
        # Repeat base data for each row in table7_data
        base_data_repeated = pd.concat([base_data] * len(table7_data), ignore_index=True)
        
        try:
            # Concatenate base data with Table 3 data along columns
            merged_df = pd.concat([base_data_repeated, table7_data], axis=1, sort=False)
            
            # Ensure 'point_of_interconnection' is present and correctly populated
            if 'point_of_interconnection' not in merged_df.columns:
                merged_df['point_of_interconnection'] = base_data['point_of_interconnection'].iloc[0]
                print(f"Added 'point_of_interconnection' to merged data for {pdf_path}.", file=log_file)
            
            print(f"Merged base data with Table 3 data for {pdf_path}.", file=log_file)
            return merged_df
        except Exception as e:
            print(f"Error merging base data with Table 3 data for {pdf_path}: {e}", file=log_file)
            print(traceback.format_exc(), file=log_file)
            return base_data  # Fallback to base data only
'''

def extract_table7_and_replace_none(pdf_path, project_id, log_file, is_addendum=False):
    """
    Extracts Table 3 data and merges with base data.
    Returns:
      df       – either base_data or base_data×Table4 rows merged
      status   – one of "no_marker", "failed", or "success"
    """
    # 1) Pull out base data
    base_data   = extract_base_data(pdf_path, project_id, log_file)
    # 2) Did we even see a Table 3 marker in the text?
    has_marker  = check_has_table7(pdf_path)
    if not has_marker:
        print(f"No Table 3 marker found in {os.path.basename(pdf_path)}; skipping extraction.", 
              file=log_file)
        return base_data, "no_marker"

    # 3) Try to scrape Table 3
    table7_data = extract_table7(pdf_path, log_file, is_addendum)
    if table7_data.empty:
        print(f"Table 3 marker found in {os.path.basename(pdf_path)}, "
              f"but extraction returned empty DataFrame.", file=log_file)
        return base_data, "failed"

    # 3) We got actual rows → merge and return
    #    Drop any overlapping columns first
    overlapping = base_data.columns.intersection(table7_data.columns)
    if not overlapping.empty:
        table7_data = table7_data.drop(columns=overlapping, errors="ignore")

    #    Repeat base_data for each row of table7_data
    base_rep   = pd.concat([base_data] * len(table7_data), ignore_index=True)
    merged_df  = pd.concat([base_rep, table7_data.reset_index(drop=True)], axis=1, sort=False)

    print(f"Merged base data with {len(table7_data)} row(s) of Table 3 for "
          f"{os.path.basename(pdf_path)}.", file=log_file)
    return merged_df, "success"





def check_has_table7(pdf_path):
    """Checks if the PDF contains Table 3-1 to 3-3."""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text = page.extract_text() or ""
                if re.search(r"Table\s*[C-D][\.-]1\b", text, re.IGNORECASE):
                    return True
    except Exception as e:
        # Handle potential errors when opening PDF
        return False
    return False

def has_network_upgrade_type_column(pdf_path, log_file):
    """Checks if any table in the PDF has a column header 'Network Upgrade Type'."""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page_number, page in enumerate(pdf.pages, start=1):
                tables = page.find_tables()
                for table_index, table in enumerate(tables, start=1):
                    tab = table.extract()
                    if not tab:
                        continue
                    headers = clean_column_headers(tab[0])
                    if "network upgrade type" in headers:
                        print(f"Found 'Network Upgrade Type' in PDF {pdf_path} on page {page_number}, table {table_index}.", file=log_file)
                        return True
    except Exception as e:
        print(f"Error checking 'Network Upgrade Type' in {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
    return False

def is_addendum(pdf_path, log_file):
    """Checks if the PDF is an addendum by searching 'Addendum' or 'Revision' on the first page."""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            if len(pdf.pages) == 0:
                return False
            first_page = pdf.pages[0]
            text = first_page.extract_text() or ""
            print(f"Extracted Text: {text}", file= log_file)  # Debugging step
            # Case-insensitive check for 'Addendum' or 'Revision'
            text_lower = text.lower()
            return "addendum" in text_lower or "revision" in text_lower
    except Exception as e:
        # Handle potential errors when opening PDF
        return False

def make_unique_headers(headers):
    """
    Appends a suffix to duplicate headers to make them unique.

    Args:
        headers (list): List of column headers.

    Returns:
        list: List of unique column headers.
    """
    seen = {}
    unique_headers = []
    for header in headers:
        if header in seen:
            seen[header] += 1
            unique_headers.append(f"{header}_{seen[header]}")
        else:
            seen[header] = 1
            unique_headers.append(header)
    return unique_headers

def process_pdfs_in_folder():
    """Processes all PDFs in the specified project range and directory."""
    global core_originals, core_addendums, total_pdfs_accessed, total_pdfs_scraped, total_pdfs_skipped, total_pdfs_skipped_extraction

    SKIP_PROJECTS = {1860, 2003, 2006}

    # Ensure the log file directory exists
    os.makedirs(os.path.dirname(LOG_FILE_PATH), exist_ok=True)

    with open(LOG_FILE_PATH, 'w') as log_file:


        for project_id in projects_to_process:
            
            # Skip the projects in the SKIP_PROJECTS set
            if project_id in SKIP_PROJECTS:
                print(f"Skipping Project {project_id} (marked to skip)", file=log_file)
                continue

         
            project_path = os.path.join(BASE_DIRECTORY, str(project_id), "03_phase_2_study")
            if not os.path.exists(project_path):
                missing_projects.add(project_id)
                print(f"Project path does not exist: {project_path}", file=log_file)
                continue

            project_scraped = False  # Flag to track if any PDF in the project was scraped
            base_data_extracted = False
            base_data = pd.DataFrame()

            # **START OF CHANGES**
            # Separate PDFs into originals and addendums
            list_pdfs = [pdf for pdf in os.listdir(project_path) if pdf.endswith(".pdf")]
            originals = []
            addendums = []
            for pdf_name in list_pdfs:
                pdf_path = os.path.join(project_path, pdf_name)
                if is_addendum(pdf_path, log_file):
                    addendums.append(pdf_name)
                else:
                    originals.append(pdf_name)
            # **END OF CHANGES**

            # **START OF CHANGES**
            # Process original PDFs first
            for pdf_name in originals:
                
                pdf_path = os.path.join(project_path, pdf_name)
                total_pdfs_accessed += 1

                is_add = is_addendum(pdf_path, log_file)

                # Check if PDF has 'Network Upgrade Type' column
                if has_network_upgrade_type_column(pdf_path, log_file):
                    style_n_pdfs.append(pdf_name)
                    print(f"Skipping PDF: {pdf_name} from Project {project_id} (Style N)", file=log_file)
                    # Still check if original has table7
                    has_table7 = check_has_table7(pdf_path)
                    original_has_table7[project_id] = has_table7
                    continue  # Skip processing this PDF

                print(f"Accessing Original PDF: {pdf_name} from Project {project_id}", file=log_file)
                original_pdfs.append(pdf_name)

                try:
                    has_table7 = check_has_table7(pdf_path)
                    original_has_table7[project_id] = has_table7

                    if not has_table7:
                        skipped_pdfs.append(pdf_name)
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} (No Table 3)", file=log_file)
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} (No Table 3)")
                        total_pdfs_skipped += 1
                        continue

                    if not base_data_extracted:
                        # Extract base data from original PDF
                        base_data = extract_base_data(pdf_path, project_id, log_file)
                        base_data_extracted = True
                        print(f"Extracted base data from original PDF: {pdf_name}", file=log_file)

                    # Extract Table 3 and merge
                    '''
                    df = extract_table7_and_replace_none(pdf_path, project_id, log_file, is_addendum=False)
                    if not df.empty:
                        core_originals = pd.concat([core_originals, df], ignore_index=True)
                        scraped_pdfs.append(pdf_name)
                        scraped_projects.add(project_id)
                        project_scraped = True
                        total_pdfs_scraped += 1
                        print(f"Scraped PDF: {pdf_name} from Project {project_id}")
                    else:
                        skipped_pdfs.append(pdf_name)
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} (Empty Data)", file=log_file)
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} (Empty Data)")
                        total_pdfs_skipped += 1
                    '''
                        # Extract Table 3 and merge
                    df, status = extract_table7_and_replace_none(pdf_path, project_id, log_file, is_addendum=False)

                    if status == "success":
                        core_originals = pd.concat([core_originals, df], ignore_index=True)
                        scraped_pdfs.append(pdf_name)
                        scraped_projects.add(project_id)
                        project_scraped = True
                        total_pdfs_scraped += 1
                        print(f"Scraped PDF: {pdf_name} from Project {project_id}")

                    elif status == "failed":
                        skipped_pdfs.append(pdf_name)
                        total_pdfs_skipped_extraction += 1
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} (Table 3 found but extraction failed)"
                             )

                    else:  # status == "no_marker"
                        skipped_pdfs.append(pdf_name)
                        total_pdfs_skipped += 1
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} (No Table 3 present)" )


                except Exception as e:
                    skipped_pdfs.append(pdf_name)
                    print(f"Skipped PDF: {pdf_name} from Project {project_id} due to an error: {e}", file=log_file)
                    print(traceback.format_exc(), file=log_file)
                    print(f"Skipped PDF: {pdf_name} from Project {project_id} due to an error: {e}")
                    total_pdfs_skipped += 1
            # **END OF CHANGES**

            # **START OF CHANGES**
            # Then process addendum PDFs
            for pdf_name in addendums:
                pdf_path = os.path.join(project_path, pdf_name)
                total_pdfs_accessed += 1
                is_add = is_addendum(pdf_path, log_file)

                # Check if PDF has 'Network Upgrade Type' column
                if has_network_upgrade_type_column(pdf_path, log_file):
                    style_n_pdfs.append(pdf_name)
                    print(f"Skipping PDF: {pdf_name} from Project {project_id} (Style N)", file=log_file)
                    continue  # Skip processing this PDF

                print(f"Accessing Addendum PDF: {pdf_name} from Project {project_id}", file=log_file)
                addendum_pdfs.append(pdf_name)

                try:
                    has_table7 = check_has_table7(pdf_path)

                    if not has_table7:
                        if original_has_table7.get(project_id, False):
                            # Attempt to scrape alternative tables is no longer needed
                            # According to the latest request, alternative table scraping is removed
                            # Therefore, we skip addendum PDFs that do not have Table 3
                            skipped_pdfs.append(pdf_name)
                            print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (No Table 3)", file=log_file)
                            print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (No Table 3)")
                            total_pdfs_skipped += 1
                        else:
                            skipped_pdfs.append(pdf_name)
                            print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (No Table 3 and original does not have Table 3)", file=log_file)
                            print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (No Table 3 and original does not have Table 3)")
                            total_pdfs_skipped += 1
                        continue

                    if not is_add and not base_data_extracted:
                        # Extract base data from original PDF
                        base_data = extract_base_data(pdf_path, project_id, log_file)
                        base_data_extracted = True
                        print(f"Extracted base data from original PDF: {pdf_name}", file=log_file)

                    if is_add and base_data_extracted:
                        # For addendums, use the extracted base data
                        table7_data = extract_table7(pdf_path, log_file, is_addendum=is_add)
                        if table7_data.empty and original_has_table7.get(project_id, False):
                            # Scrape alternative tables is removed, so skip if no data
                            skipped_pdfs.append(pdf_name)
                            print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (No relevant tables found)", file=log_file)
                            print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (No relevant tables found)")
                            total_pdfs_skipped += 1
                        if not table7_data.empty:
                            # Merge base data with Table 3 data
                            merged_df = pd.concat([base_data] * len(table7_data), ignore_index=True)
                            merged_df = pd.concat([merged_df, table7_data], axis=1, sort=False)
                            core_addendums = pd.concat([core_addendums, merged_df], ignore_index=True)
                            scraped_pdfs.append(pdf_name)
                            scraped_projects.add(project_id)
                            project_scraped = True
                            total_pdfs_scraped += 1
                            print(f"Scraped Addendum PDF: {pdf_name} from Project {project_id}")
                        else:
                            skipped_pdfs.append(pdf_name)
                            print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (Empty Data)", file=log_file)
                            print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (Empty Data)")
                            total_pdfs_skipped += 1
                except Exception as e:
                    skipped_pdfs.append(pdf_name)
                    print(f"Skipped PDF: {pdf_name} from Project {project_id} due to an error: {e}", file=log_file)
                    print(traceback.format_exc(), file=log_file)
                    # Optionally, print to ipynb
                    print(f"Skipped PDF: {pdf_name} from Project {project_id} due to an error: {e}")
                    total_pdfs_skipped += 1
            # **END OF CHANGES**

            # After processing all PDFs for this project, check if any PDF was scraped
            if not project_scraped and os.path.exists(project_path):
                skipped_projects.add(project_id)

    # Rest of the code remains unchanged...

    # After processing all PDFs, save to CSV
    save_to_csv(core_originals, OUTPUT_CSV_PATH_ORIGINAL, "originals")
    save_to_csv(core_addendums, OUTPUT_CSV_PATH_ADDENDUM, "addendums")

    # Calculate total projects processed
    total_projects_processed = len(scraped_projects) + len(skipped_projects)

    # Print summary to ipynb
    print("\n=== Scraping Summary ===")
    print(f"Total Projects Processed: {total_projects_processed}")
    print(f"Total Projects Scraped: {len(scraped_projects)}")
    print(f"Total Projects Skipped due to failed extraction of Table: {total_pdfs_skipped_extraction}")
    print(f"Total Projects Skipped: {len(skipped_projects)}")
    print(f"Total Projects Missing: {len(missing_projects)}")
    print(f"Total PDFs Accessed: {total_pdfs_accessed}")
    print(f"Total PDFs Scraped: {total_pdfs_scraped}")
    print(f"Total PDFs Skipped: {total_pdfs_skipped}")

    print("\nList of Scraped Projects:")
    print(sorted(scraped_projects))

    print("\nList of Skipped Projects:")
    print(sorted(skipped_projects))

    print("\nList of Missing Projects:")
    print(sorted(missing_projects))

    print("\nList of Scraped PDFs:")
    print(scraped_pdfs)

    print("\nList of Skipped PDFs:")
    print(skipped_pdfs)

    print("\nList of Addendum PDFs:")
    print(addendum_pdfs)

    print("\nList of Original PDFs:")
    print(original_pdfs)

    print("\nList of Style N PDFs (Skipped due to 'Network Upgrade Type'):")
    print(style_n_pdfs)

    print("\nTotal Number of Style N PDFs:", len(style_n_pdfs))

    print("\nNumber of Original PDFs Scraped:", len([pdf for pdf in scraped_pdfs if pdf in original_pdfs]))
    print("Number of Addendum PDFs Scraped:", len([pdf for pdf in scraped_pdfs if pdf in addendum_pdfs]))

def save_to_csv(df, output_csv_path, data_type):
    """Cleans the DataFrame and saves it to a CSV file."""
    if df.empty:
        print(f"No data to save for {data_type}.")
        return

    # Clean up the entire DataFrame by cleaning string cells
    df = df.applymap(clean_string_cell)

    # Drop rows that contain specific phrases (e.g., "Type of Upgrade")
    df = df[~df.apply(lambda row: contains_phrase(row, "Type of Upgrade"), axis=1)]

    # Reorder columns as specified
    df = reorder_columns(df)
    print(f"\nColumns reordered for {data_type} as per specification.")

    # Ensure q_id is numeric for sorting, replace missing values with None
 

    # Save the DataFrame to CSV
    try:
        df.to_csv(output_csv_path, index=False)
        print(f"\nData successfully saved to {output_csv_path}")
    except Exception as e:
        print(f"Error saving {data_type} data to CSV: {e}")
        print(traceback.format_exc())

def main():
    """Main function to execute the PDF scraping process."""
    process_pdfs_in_folder()

if __name__ == "__main__":
    main()


Will process these project‐folders:
['643', '643AS', '643AA', '643AF', '643AH', '643AI', '643I', '643G', '643R', '643F', '643O', '643Z', '643S', '643T', '643AE', '643AB', '643AK', '643AP', '643AM', '643AJ', '643AC', '643X', '643J', '643D', '643W', '643E']
Skipped PDF: C3C4P2-Northern-Q643AA-Blue Sky Wind Energy Center-AppendixA.pdf from Project 643AA (Table 3 found but extraction failed)
Skipped Addendum PDF: 10AS665623-QC34PIINorthernQ643AAAddendum_20121227.pdf from Project 643AA (No relevant tables found)
Skipped Addendum PDF: 10AS665623-QC34PIINorthernQ643AAAddendum_20121227.pdf from Project 643AA (Empty Data)


KeyboardInterrupt: 

# Itemized and Addendums

In [ ]:
import pandas as pd
import re
import unicodedata

# Load the CSV file
df = pd.read_csv("/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/03_raw/ph2_rawdata_cluster3_style_new_originals.csv", dtype={'estimated_time_to_construct': str})

df['q_id'] = df['q_id'].astype('Int64')
df['cluster'] = df['cluster'].astype('Int64')


def clean_column_headers(headers):
    """Cleans column headers by normalizing and removing unwanted characters."""
    cleaned_headers = []  # Initialize an empty list to hold the cleaned header names.
    for header in headers:  # Iterate over each header in the input.
        if header is None:
            header = ""  # If the header is None, set it to an empty string.
        elif isinstance(header, str):  # Otherwise, if the header is a string:
            #header = header.lower()  # Convert the header to lowercase.
            header = re.sub(r'\s+', ' ', header)  # Replace one or more whitespace characters with a single space.
            #header = re.sub(r'\(.*?\)', '', header)  # Remove any text within parentheses (non-greedy).
            header = re.sub(r'[^a-zA-Z0-9\s()/+=_]', '', header)  # Remove any character that is not a letter, number, or whitespace.
            header = header.strip()  # Remove any leading or trailing whitespace.
        cleaned_headers.append(header)  # Append the cleaned header to the list.
    return cleaned_headers  # Return the list of cleaned headers.



#df.columns = clean_column_headers(df.columns)

def convert_to_snake_case(column_name):
    column_name = column_name.strip().lower()
    column_name = re.sub(r'[\s\-]+', '_', column_name)
    column_name = re.sub(r'[^\w]', '', column_name)
    return column_name

def clean_string_cell(value):
    if isinstance(value, str):
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
        value = value.replace('\n', ' ').strip()
    return value

#df = df.map(clean_string_cell)
#df.columns = [convert_to_snake_case(col) for col in df.columns]
print("After cleaning:", df.columns.tolist())

# Q945 PTO first and only row esc cost 41.3 is entered in the pdf as 41,3, have to manually update.

# Originals

In [15]:
import pandas as pd
import re
import unicodedata

# Load the CSV file
df = pd.read_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/03_raw/ph2_rawdata_cluster3_style_new_originals.csv', dtype={'estimated_time_to_construct': str})

#df['q_id'] = df['q_id'].astype('Int64')
df['cluster'] = df['cluster'].astype('Int64')


######################################################################################################################################
########################################
#STEP 1 MERGE COLUMNS

def merge_columns(df):
    merge_columns_dict = {
        "escalated_cost_x_1000": [
            "escalated costs x 1000",
            'escalated costs x 1000 (note 1)',
            'estimated cost x 1000 (od year) (note 1)',
            "estimated cost x 1000 escalated (note 1)",
            "estimated cost x 1000 escalated",
            'Esc',
            "estimated cost x 1000 escalated without itcca",
             "estimated cost x 1000 escalated without itcca (note 1)",
            "estimated cost x 1000 (od year) (note 1)",
             "estimated cost x 1000 (od year) (note 1)",
             "estimated_cost_x_1000_escalated_note_1",
            "escalated cost x 1000",
            "sum of allocated escalated cost",
            
        ],
        "estimated_cost_x_1000": [
            "estimated cost x 1000",
            "allocated_cost",
            "sum of allocated constant cost"
        ],
        "estimated_time_to_construct": [
            "estimated time to construct",
            "estimated time  to construct",
            "estimated time to construct (note 1)",
            "estimated time to construct(note 1)",
            "estimated time (months) to construct (note 1)",
            'estimated time (months) to construct (note 2)',
            "estimated time to construct (note 3)",
            "estimated time to construct (note 2)",
            'estimated time (months) to execute(note 2)', 
            'estimated time (months) to execute (note 2)', 
            'estimated time (months) to construct (note 1)',
        ],
        "total_estimated_cost_x_1000_escalated": [
            "total estimated cost x 1000 escalalted",
            "total estimated cost x 1000 escalated"
        ],
        "total_estimated_cost_x_1000": [
            "total nu cost",
            "total cost constant"
        ],
        "adnu_cost_rate_x_1000": [
            "adnu cost rate x 1000",
            "cost rate x 1000",
            "cost rate",
            "cost rate constant"
        ],
        "description": ["description", "description installation of dtt receivers"],
        "capacity": [
            "capacity",
            "project size",
            "project mw",
            "project size (mw)"
            "mw at poi"
        ],
        "cost_allocation_factor": [
            "cost allocation factor",
            "cost allocatio n factor",
            "cost allocati on factor",
        ],
    }

    # Identify unnamed columns
    unnamed_columns = [col for col in df.columns if pd.isna(col) or col.strip() == "" or col.startswith("Unnamed")]
    if unnamed_columns:
        merge_columns_dict["description"].extend(unnamed_columns)

    for new_col, old_cols in merge_columns_dict.items():
        existing_cols = [col for col in old_cols if col in df.columns]
        if existing_cols:
            df[new_col] = df[existing_cols].bfill(axis=1).iloc[:, 0]
            cols_to_drop = [col for col in existing_cols if col != new_col]
            if cols_to_drop:
                df.drop(columns=cols_to_drop, inplace=True)

    return df

df = merge_columns(df)
#df.drop('incremental deliverability', axis=1, inplace=True)
#df.drop('dependent system upgrade', axis=1, inplace=True)
#df.drop('upgrade_classification', axis=1, inplace=True)

#STEP 2: NAMING CONVENTION
def convert_to_snake_case(column_name):
    column_name = column_name.strip().lower()
    column_name = re.sub(r'[\s\-]+', '_', column_name)
    column_name = re.sub(r'[^\w]', '', column_name)
    return column_name

def clean_string_cell(value):
    if isinstance(value, str):
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
        value = value.replace('\n', ' ').strip()
    return value

df = df.map(clean_string_cell)
df.columns = [convert_to_snake_case(col) for col in df.columns]
 

required_columns = ['type_of_upgrade', 'cost_allocation_factor']
for col in required_columns:
    if col not in df.columns:
        df[col] = None


df = df[
    ~df['type_of_upgrade'].str.contains(r'Precursor Network Upgrades \(PNU\)|Estimated in Service Date', na=False)
]

df = df[~df.apply(lambda row: row.astype(str).isin(["Total Of This", "of this"]).any(), axis=1)]
 
 
# Step 3: Rename 'Grand Total' to 'Total' in total_estimated_cost_x_1000
if 'total_estimated_cost_x_1000' in df.columns:
    df['total_estimated_cost_x_1000'] = df['total_estimated_cost_x_1000'].replace('Grand Total', 'Total')

# Step 3: Move 'Total' from total_estimated_cost_x_1000 to cost_allocation_factor
if 'total_estimated_cost_x_1000' in df.columns and 'cost_allocation_factor' in df.columns:
    df['cost_allocation_factor'] = df.apply(
        lambda row: 'Total' if str(row['total_estimated_cost_x_1000']) == 'Total' else row['cost_allocation_factor'],
        axis=1
    )
    df['total_estimated_cost_x_1000'] = df['total_estimated_cost_x_1000'].replace('Total', None)


 

df = df[df["description"] != "Total Allocated"]    

if 'description' in df.columns and 'cost_allocation_factor' in df.columns:
    df['cost_allocation_factor'] = df.apply(
        lambda row: 'Total' if str(row['description']) == 'Total' else row['cost_allocation_factor'],
        axis=1
    )
    df['description'] = df['description'].replace('Total', None)

mask_agg = (
    df['type_of_upgrade'].fillna('').eq('Total') |
    df['cost_allocation_factor'].fillna('').eq('Total')
)

# 2) Extract them
aggregate_total = df.loc[mask_agg].copy()

# 3) Tag them in the original df
df['is_aggregate_total'] = mask_agg


agg_data = df[df['is_aggregate_total']].copy()
agg_data.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/02_intermediate/costs_phase_2_cluster_3_style_new_aggregate.csv', index=False) 

# 3) Then drop them from your main itemized set
df = df.loc[~mask_agg].reset_index(drop=True)

df.drop(columns=['is_aggregate_total'], inplace=True, errors='ignore')


df['item'] = df.apply(
    lambda row: 'no' if (
        (pd.notna(row.get('type_of_upgrade')) and 'Total' in str(row['type_of_upgrade'])) or
        (pd.notna(row.get('cost_allocation_factor')) and 'Total' in str(row['cost_allocation_factor']))
    ) else 'yes',
    axis=1
)
   
#df.drop( [ 'estimated_cost_x_1000_escalated_with_itcca_note_2',	 	'estimated_cost_x_1000_escalated_with_itcca_note_2' ], axis=1, inplace=True)

# Step 3: Clean the type of upgrade column
   
 
df['type_of_upgrade'] = (
    df['type_of_upgrade']
    .fillna('')  # Temporarily replace NaN with an empty string
    .str.replace(r'\(Note \d+\)', '', regex=True)  # Remove (Note digit)
    .str.strip()  # Strip leading/trailing whitespace
    .str.title()  # Capitalize the first letter of each word
    .str.replace(r'Upgrades$', 'Upgrade', regex=True)  # Fix plural endings
    .replace('', pd.NA)  # Convert empty strings back to NaN
)

 


df.dropna(subset=['upgrade'], inplace=True)


mappings = {
    'Reliability Network Upgrade': 'RNU',
    'Reliability Network Upgrades': 'RNU',
    'Local Delivery Network Upgrade': 'LDNU',
    'Local Delivery Network': 'LDNU',
    'Potential Local Delivery Network Upgrade': 'LDNU',
    "Ptos Interconnection Facilities": 'PTO_IF',
    'Escalated Cost and Time to Construct for Interconnection Facilities, Reliability Network Upgrades, and Delivery Network': 'LDNU',
    'Escalated Cost And Time To Construct For Interconnection Facilities, Reliability Network upgrade, And Delivery Network': 'LDNU',
    'Escalated Cost And Time To Construct For Interconnection Facilities, Reliability Network upgrade, And Delivery': 'LDNU',
    'Deliverability Network Upgrade': 'LDNU',
    'Delivery Network Upgrade': 'LDNU',
    'Escalated Cost And Time To Construct For Interconnection Facilities - If': 'PTO_IF',
    'Area Delivery Network Upgrade': 'ADNU',
    'Reliability Network Upgrade To Physically Interconnect': 'RNU',
    "Reliability Network upgrade To Physically Interconnect": "RNU",
     'Pto': 'PTO_IF',
    "Other Potential Network Upgrade": "OPNU",
    "Conditionally Assigned Network Upgrade": "CANU",
    "Canus": "CANU",
    'Local  Delivery  Network  Upgrade': 'LDNU',
        "Escalated Cost And Time To Construct For Reliability Network Upgrade4": "RNU",
    "Escalated Cost And Time To Construct For Reliability Network Upgrade3": "RNU",
    'Escalated Cost And Time To Construct For Reliability Network Upgrade': "RNU",
    'Rnus, Estimated Costs, And Estimated Time To Construct Summary': "RNU",
    "Local Off-Peak Network Upgrade": "LOPNU",
    'Total PTO_IF': 'PTO_IF',
 'Total RNU': 'RNU',
 'Total LDNU': 'LDNU',
 'Total OPNU' : 'OPNU',
 'Total CANU': 'CANU',
 'Total LOPNU': 'LOPNU',
 'Canu': 'CANU',
  
 'Total ADNU': 'ADNU',
  'Ptos Interconnect Ion Facilities' : 'PTO_IF',
  'Local Off- Peak Network Upgrade': 'LOPNU',
 'P Os Interconnection Facilities': 'PTO_IF',
  

 
}


#Step 3: Apply mapping and ffill type of upgrade column

if 'type_of_upgrade' in df.columns:
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: re.sub(r'\bupgrades\b', 'upgrade', x, flags=re.IGNORECASE).strip() if isinstance(x, str) else x
    )
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: mappings.get(x, x) if isinstance(x, str) else x
    )
    df['type_of_upgrade'] = df['type_of_upgrade'].ffill()  



import re
import pandas as pd


# build a boolean mask: True for rows where cost contains "(…)"
has_paren = df['estimated_cost_x_1000'].astype(str).str.contains(r'\([^)]*\)')
# drop those rows
df = df.loc[~has_paren].copy()

def drop_rows_with_star_in_costs(df, cost_cols):
    """
    Remove any row where any of the cost_cols contains a '*' character.
    """
    mask = pd.Series(False, index=df.index)
    for col in cost_cols:
        if col in df.columns:
            # star anywhere in the string
            mask = mask | df[col].astype(str).str.contains(r"\*", regex=True)
    # keep only rows without a star
    return df.loc[~mask].reset_index(drop=True)


import re

def replace_star_costs_with_zero(df, cost_cols):
    """
    Zero out any starred cost entries, then coerce remaining strings to floats.
    Handles multiple dots by removing all but the final one (so "1.365.9" → "1365.9").
    """
    def normalize_num_string(s: str) -> float:
        # Remove all but the last dot:
        #   "1.365.9" → first remove the dot before the final one → "1365.9"
        s = re.sub(r'\.(?=.*\.)', '', s)
        # Strip out everything except digits, dot, and minus
        s = re.sub(r'[^0-9\.\-]', '', s)
        # If it’s now just empty or just a dot/minus, treat as zero
        if re.fullmatch(r'[-\.]*', s):
            return 0.0
        try:
            return float(s)
        except ValueError:
            return 0.0

    for col in cost_cols:
        if col not in df.columns:
            continue

        # 1) Zero out any starred entries
        star_mask = df[col].astype(str).str.contains(r'\*', regex=True, na=False)
        df.loc[star_mask, col] = "0"

        # 2) Normalize & convert every cell in this column
        df[col] = (
            df[col]
            .astype(str)
            .apply(normalize_num_string)
        )
    return df





def drop_rows_with_dash_in_time(df, time_col):
    """
    Remove any row where the time_col contains a dash '-' (e.g. '-').
    """
    if time_col in df.columns:
        mask = df[time_col].astype(str).str.contains(r"^-+$", regex=True)
        return df.loc[~mask].reset_index(drop=True)
    return df

def keep_second_entry_in_cells(df, columns):
    """
    For each column in columns, if the cell contains multiple space‑separated entries,
    keep only the second one.
    """
    for col in columns:
        if col not in df.columns:
            continue
        def pick_second(cell):
            parts = re.findall(r"[\d\.\$%,]+", str(cell))
            return parts[1] if len(parts) > 1 else (parts[0] if parts else cell)
        df[col] = df[col].apply(pick_second)
    return df

# ── Integration ──
# Place this just before your Step 7 clean_currency block:

# 1) drop any row where estimated or escalated cost has '*'
df = replace_star_costs_with_zero(
    df,
    cost_cols=[
        'estimated_cost_x_1000',
        'escalated_cost_x_1000',
        'total_estimated_cost_x_1000',
        'total_estimated_cost_x_1000_escalated'
    ]
)


# 2) drop any row where estimated_time_to_construct is just a dash
df = drop_rows_with_dash_in_time(df, 'estimated_time_to_construct')

 
# Now proceed with your clean_currency step…


    
# Step 3: Remove $ signs and convert to numeric
import re

def clean_currency(value):
    """
    Cleans a string by explicitly removing $, *, (Note 2), and similar patterns,
    then converts it to a numeric value.
    """
    if isinstance(value, str):
        # Explicitly remove $, *, and any "(Note ...)"
        value = value.replace('$', '').replace('*', '')
        value = re.sub(r'\(Note \d+\)', '', value)  # Remove patterns like "(Note 2)"
        value = value.replace(',', '').strip()  # Remove commas and extra spaces
    try:
        return pd.to_numeric(value)
    except ValueError:
        return pd.NA  # Return NaN for invalid entries


# Clean the specific columns
for col in ['estimated_cost_x_1000', 'escalated_cost_x_1000', 'total_estimated_cost_x_1000_escalated', 'total_estimated_cost_x_1000', 'adnu_cost_rate_x_1000', 'adnu_cost_rate_x_1000_escalated']:
    if col in df.columns:
        df[col] = df[col].apply(clean_currency)

# Create Total rows
new_rows = []
columns_to_sum = ['estimated_cost_x_1000', 'escalated_cost_x_1000', 'total_estimated_cost_x_1000_escalated',   'adnu_cost_rate_x_1000_escalated']
columns_to_populate = ['cluster', 'req_deliverability', 'latitude', 'longitude', 'capacity', 'point_of_interconnection']

for q_id, group in df.groupby('q_id', as_index=False):
    unique_upgrades = group['type_of_upgrade'].dropna().unique()
    for upgrade in unique_upgrades:
        if pd.isna(upgrade):
            continue
        
        rows = group[group['type_of_upgrade'] == upgrade]

                # Check if a Total row already exists for this (q_id, upgrade)
        total_exists = group[
            (group['type_of_upgrade'] == upgrade) & (group['item'] == 'no')
        ].shape[0] > 0
        
        if total_exists:
             
            continue


 
        
        
        if not total_exists:
            # If only one row exists, duplicate it as the total row
            if len(rows) == 1:

                total_row = {col: '' for col in df.columns}  # Initialize all columns as empty strings

                # Populate the necessary fields
                total_row['q_id'] = q_id
                total_row['type_of_upgrade'] = f"Total {upgrade}"
                total_row['item'] = 'no'

                # Populate specified columns from the existing row
                first_row = rows.iloc[0]
                for col in columns_to_populate:
                    if col in df.columns:
                        total_row[col] = first_row[col]

                # Sum the numeric columns (single row, so it remains the same)
                for col in columns_to_sum:
                    if col in rows.columns:
                        total_row[col] = rows[col].sum()
                    else:
                        total_row[col] = 0  # Default to 0 if column is missing

                new_rows.append(total_row)



 
            
            # If multiple rows exist, sum numeric columns and create a total row
            elif len(rows) > 1:
                total_row = {col: '' for col in df.columns}  # Initialize all columns as empty strings

                # Populate the necessary fields
                total_row['q_id'] = q_id
                total_row['type_of_upgrade'] = f"Total {upgrade}"
                total_row['item'] = 'no'

                # Populate the specified columns from the first row in the group
                first_row = rows.iloc[0]
                for col in columns_to_populate:
                    if col in df.columns:
                        total_row[col] = first_row[col]

                # Sum the numeric columns
                for col in columns_to_sum:
                    if col in rows.columns:
                        total_row[col] = rows[col].sum()
                    else:
                        total_row[col] = 0  # Default to 0 if column is missing

                new_rows.append(total_row)
 



      

if new_rows:
    total_rows_df = pd.DataFrame(new_rows)
    for col in df.columns:
        if col not in total_rows_df.columns:
            total_rows_df[col] = None
    total_rows_df = total_rows_df[df.columns]
    df = pd.concat([df, total_rows_df], ignore_index=True)

df.reset_index(drop=True, inplace=True)

# Update 'item' column based on Total in type_of_upgrade or cost_allocation_factor
df['item'] = df.apply(
    lambda row: 'no' if (
        'Total' in str(row.get('type_of_upgrade', '')) or 
        'Total' in str(row.get('cost_allocation_factor', ''))
    ) else 'yes',
    axis=1
)


# Step 3: Move 'item' column next to 'type_of_upgrade'
if 'item' in df.columns and 'type_of_upgrade' in df.columns:
    cols = df.columns.tolist()
    item_index = cols.index('item')
    type_index = cols.index('type_of_upgrade')
    if item_index < type_index:
        cols.insert(type_index + 1, cols.pop(item_index))
    else:
        cols.insert(type_index + 1, cols.pop(item_index))
    df = df[cols]

# Step 9: Remove "Total" values from cost_allocation_factor if they appear in type_of_upgrade
if 'cost_allocation_factor' in df.columns and 'type_of_upgrade' in df.columns:
    df['cost_allocation_factor'] = df.apply(
        lambda row: None if (
            pd.notna(row['type_of_upgrade']) and 'Total' in str(row['type_of_upgrade'])
        ) else row.get('cost_allocation_factor'),
        axis=1
    )
    

if 'cost_allocation_factor' in df.columns and 'type_of_upgrade' in df.columns:
    df['cost_allocation_factor'] = df.apply(
        lambda row: None if 'Total' in str(row.get('cost_allocation_factor', '')) else row.get('cost_allocation_factor'),
        axis=1
    )



def clean_estimated_time(value):
    if isinstance(value, str):
        value = re.sub(r'(\d+(?:-\w+)*)\s+\w+.*$', r'\1', value, flags=re.IGNORECASE).strip()
    return value

if 'estimated_time_to_construct' in df.columns:
    df['estimated_time_to_construct'] = df['estimated_time_to_construct'].apply(clean_estimated_time)

if 'type_of_upgrade' in df.columns:
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: re.sub(r'\s*\(note \d+\)', '', x, flags=re.IGNORECASE).strip() if isinstance(x, str) else x
    )

 



if 'type_of_upgrade' in df.columns:
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: re.sub(r'\bupgrades\b', 'upgrade', x, flags=re.IGNORECASE).strip() if isinstance(x, str) else x
    )
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: mappings.get(x, x) if isinstance(x, str) else x
    )
    df['type_of_upgrade'] = df['type_of_upgrade'].ffill()

if 'upgrade' in df.columns and 'type_of_upgrade' in df.columns and 'q_id' in df.columns:
    df['upgrade'] = df.groupby(['q_id', 'type_of_upgrade'])['upgrade'].ffill()



if 'type_of_upgrade' in df.columns:
    previous_type_of_upgrade = None
    for i in range(len(df)):
        val = df.at[i, 'type_of_upgrade']
        # only do the equality check if val is not NA
        if pd.notna(val) and val == 'Total':
            if previous_type_of_upgrade is not None:
                df.at[i, 'type_of_upgrade'] = previous_type_of_upgrade
        else:
            previous_type_of_upgrade = val

df.dropna(subset=['type_of_upgrade'], inplace=True)



numeric_columns = [
    'cost_allocation_factor',
    'estimated_cost_x_1000',
    'estimated_time_to_construct',
    'total_estimated_cost_x_1000_escalated',
    'adnu_cost_rate_x_1000',
    'escalated_cost_x_1000',
    'estimated_cost_x_1000_escalated_without_itcca',
    'adnu_cost_rate_x_1000_escalated'
]
non_numeric_columns = ['type_of_upgrade', 'upgrade', 'description']

for col in non_numeric_columns:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: 'None' if pd.isna(x) else x)

for col in numeric_columns:
    if col in df.columns:
        df[col] = df[col].replace('-', pd.NA)
        df[col] = df[col].fillna(0)

if 'original_order' in df.columns and 'q_id' in df.columns:
    df['original_order'] = df.index
    df = df.sort_values(by=['q_id', 'original_order'], ascending=[True, True])
    df = df.drop(columns=['original_order'])


def reorder_columns(df):
    """
    Reorders the columns of the DataFrame based on the specified order.

    Args:
        df (pd.DataFrame): The DataFrame to reorder.

    Returns:
        pd.DataFrame: The reordered DataFrame.
    """
    desired_order = [
        "q_id",
        "cluster",
        "req_deliverability",
        "latitude",
        "longitude",
        "capacity",
        "point_of_interconnection",
        "type_of_upgrade",
        "upgrade",
        "description",
        "cost_allocation_factor",
        "estimated_cost_x_1000",
        "escalated_cost_x_1000",
        "total_estimated_cost_x_1000",
        "total_estimated_cost_x_1000_escalated",
        "estimated_time_to_construct",
    ]

    # Start with desired columns that exist in the DataFrame
    existing_desired = [col for col in desired_order if col in df.columns]

    # Then add the remaining columns
    remaining = [col for col in df.columns if col not in existing_desired]

    # Combine the two lists
    new_order = existing_desired + remaining

    # Reorder the DataFrame
    df = df[new_order]

    return df        


df= reorder_columns(df)

# build a mask of exactly the rows you want to drop
mask = (
    (df['type_of_upgrade'] == 'PTO_IF') &
    (df['upgrade'] == 'None') &
    (df['description'] == 'None') &
    (df['cost_allocation_factor'] == 0) &
    (df['estimated_cost_x_1000'] == 0) &
    (df['escalated_cost_x_1000'] == 0) &
    (df['estimated_time_to_construct'] == 0)
)

 

# if you want to do it in-place instead:
df.drop(df[mask].index, inplace=True)



# Save itemized and totals separately
if 'item' in df.columns:
    itemized_df = df[df['item'] == 'yes']
    itemized_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/02_intermediate/costs_phase_2_cluster_3_style_new_itemized.csv', index=False)

    totals_columns = ['upgrade', 'description', 'cost_allocation_factor', 'estimated_time_to_construct']
    totals_df = df[df['item'] == 'no'].drop(columns=totals_columns, errors='ignore')
    totals_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/02_intermediate/costs_phase_2_cluster_3_style_new_total.csv', index=False)

print(f"Itemized rows saved to 'costs_phase_2_cluster_3_style_new_itemized.csv'.")
print(f"Filtered Total rows saved to 'costs_phase_2_cluster_3_style_new_total.csv'.")


if 'type_of_upgrade' in df.columns:
    print(df['type_of_upgrade'].unique())

if 'q_id' in df.columns:
    print(df['q_id'].unique())

if 'cluster' in df.columns:
    print(df['cluster'].unique())


Itemized rows saved to 'costs_phase_2_cluster_3_style_new_itemized.csv'.
Filtered Total rows saved to 'costs_phase_2_cluster_3_style_new_total.csv'.
['PTO_IF' 'LDNU' 'RNU']
['643I' '643W']
[3]


# Addendums

In [8]:
import pandas as pd
import re
import unicodedata

# Load the CSV file
df = pd.read_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/03_raw/ph2_rawdata_cluster3_style_new_addendums.csv', dtype={'estimated_time_to_construct': str})

#df['q_id'] = df['q_id'].astype('Int64')
df['cluster'] = df['cluster'].astype('Int64')


######################################################################################################################################
########################################
#STEP 1 MERGE COLUMNS

def merge_columns(df):
    merge_columns_dict = {
        "escalated_cost_x_1000": [
            "escalated costs x 1000",
            "escalated cost x 1000 (note 1)",
            "estimated cost x 1000 escalated",
            "estimated cost x 1000 escalated",
            "estimated cost x 1000 escalated without itcca",
            "estimated cost x 1000 escalated without itcca (note 1)",
            "escalated cost x 1000",
            "sum of allocated escalated cost",
            "estimated cost x 1000 escalated (note 1)",
            "escalated costs x 1000 (note 1)",
        ],
        "estimated_cost_x_1000": [
            "estimated cost x 1000",
            "allocated_cost",
            "sum of allocated constant cost",
        ],
        "estimated_time_to_construct": [
            "estimated time to construct",
            "estimated time  to construct",
            "estimated_time_months_to_construct_note_1",
            "estimated time (months) to construct (note 1)",
            "estimated time (months) to construct (note 2)",
            "estimated time (months to construct) (note 2)",
            "estimated time to construct (note 2)",
            "estimated time to construct (note 3)",
            "estimated time (months) to execute (note 2)",
            "estimated time (months) to execute(note 2)",
        ],
        "total_estimated_cost_x_1000_escalated": [
            "total estimated cost x 1000 escalalted",
            "total estimated cost x 1000 escalated"
        ],
        "total_estimated_cost_x_1000": [
            "total nu cost",
            "total cost constant"
        ],
        "adnu_cost_rate_x_1000": [
            "adnu cost rate x 1000",
            "cost rate x 1000",
            "cost rate",
            "cost rate constant"
        ],
        "description": ["description"],
        "capacity": [
            "capacity",
            "project size",
            "project mw",
            "mw at poi"
        ],
        "cost_allocation_factor": [
            "cost allocation factor",
            "cost allocatio n factor",
            "cost allocati on factor",
        ],
    }

    # Identify unnamed columns
    unnamed_columns = [col for col in df.columns if pd.isna(col) or col.strip() == "" or col.startswith("Unnamed")]
    if unnamed_columns:
        merge_columns_dict["description"].extend(unnamed_columns)

    for new_col, old_cols in merge_columns_dict.items():
        existing_cols = [col for col in old_cols if col in df.columns]
        if existing_cols:
            df[new_col] = df[existing_cols].bfill(axis=1).iloc[:, 0]
            cols_to_drop = [col for col in existing_cols if col != new_col]
            if cols_to_drop:
                df.drop(columns=cols_to_drop, inplace=True)

    return df

df = merge_columns(df)
#df.drop('incremental deliverability', axis=1, inplace=True)
#df.drop('dependent system upgrade', axis=1, inplace=True)

#STEP 2: NAMING CONVENTION
def convert_to_snake_case(column_name):
    column_name = column_name.strip().lower()
    column_name = re.sub(r'[\s\-]+', '_', column_name)
    column_name = re.sub(r'[^\w]', '', column_name)
    return column_name

def clean_string_cell(value):
    if isinstance(value, str):
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
        value = value.replace('\n', ' ').strip()
    return value

df = df.map(clean_string_cell)
df.columns = [convert_to_snake_case(col) for col in df.columns]
 


required_columns = ['type_of_upgrade', 'cost_allocation_factor']
for col in required_columns:
    if col not in df.columns:
        df[col] = None


df = df[
    ~df['type_of_upgrade'].str.contains(r'Precursor Network Upgrades \(PNU\)|Estimated in Service Date', na=False)
]
 
 
# Step 3: Rename 'Grand Total' to 'Total' in total_estimated_cost_x_1000
if 'total_estimated_cost_x_1000' in df.columns:
    df['total_estimated_cost_x_1000'] = df['total_estimated_cost_x_1000'].replace('Grand Total', 'Total')

# Step 3: Move 'Total' from total_estimated_cost_x_1000 to cost_allocation_factor
if 'total_estimated_cost_x_1000' in df.columns and 'cost_allocation_factor' in df.columns:
    df['cost_allocation_factor'] = df.apply(
        lambda row: 'Total' if str(row['total_estimated_cost_x_1000']) == 'Total' else row['cost_allocation_factor'],
        axis=1
    )
    df['total_estimated_cost_x_1000'] = df['total_estimated_cost_x_1000'].replace('Total', None)

df = df[df["description"] != "Total Allocated"]    

if 'description' in df.columns and 'cost_allocation_factor' in df.columns:
    df['cost_allocation_factor'] = df.apply(
        lambda row: 'Total' if str(row['description']) == 'Total' else row['cost_allocation_factor'],
        axis=1
    )
    df['description'] = df['description'].replace('Total', None)


mask_agg = (
    df['type_of_upgrade'].fillna('').eq('Total') |
    df['cost_allocation_factor'].fillna('').eq('Total')
)

# 2) Extract them
aggregate_total = df.loc[mask_agg].copy()

# 3) Tag them in the original df
df['is_aggregate_total'] = mask_agg


agg_data = df[df['is_aggregate_total']].copy()
agg_data.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/02_intermediate/costs_phase_2_cluster_3_style_new_aggregate_addendums.csv', index=False) 



# 3) Then drop them from your main itemized set
df = df.loc[~mask_agg].reset_index(drop=True)


df.drop(columns=['is_aggregate_total'], inplace=True, errors='ignore')

df['item'] = df.apply(
    lambda row: 'no' if (
        (pd.notna(row.get('type_of_upgrade')) and 'Total' in str(row['type_of_upgrade'])) or
        (pd.notna(row.get('cost_allocation_factor')) and 'Total' in str(row['cost_allocation_factor']))
    ) else 'yes',
    axis=1
)
   
#df.drop(['interconnection_facility_element', 'cost_subject_to_itcc',	'total_cost_excluding_itcc_note_1'], axis=1, inplace=True)

# Step 3: Clean the type of upgrade column
   
 
df['type_of_upgrade'] = (
    df['type_of_upgrade']
    .fillna('')  # Temporarily replace NaN with an empty string
    .str.replace(r'\(Note \d+\)', '', regex=True)  # Remove (Note digit)
    .str.strip()  # Strip leading/trailing whitespace
    .str.title()  # Capitalize the first letter of each word
    .str.replace(r'Upgrades$', 'Upgrade', regex=True)  # Fix plural endings
    .replace('', pd.NA)  # Convert empty strings back to NaN
)





mappings = {
    'Reliability Network Upgrade': 'RNU',
    'Reliability Network Upgrades': 'RNU',
    'Local Delivery Network Upgrade': 'LDNU',
    'Local Delivery Network': 'LDNU',
    'Potential Local Delivery Network Upgrade': 'LDNU',
    "Ptos Interconnection Facilities": 'PTO_IF',
    'Area Delivery Network Upgrade': 'ADNU',
    'Reliability Network Upgrade To Physically Interconnect': 'RNU',
    "Reliability Network upgrade To Physically Interconnect": "RNU",
    'Escalated Cost And Time To Construct For Reliability Network Upgrade': 'RNU',
     'Escalated Cost And Time To Construct For Interconnection Facilities, Reliability Network upgrade, And Delivery': 'LDNU',
 'Delivery Network Upgrade': 'LDNU',
 'Total Escalated Cost And Time To Construct For Interconnection Facilities, Reliability Network upgrade, And Delivery': 'LDNU',
     'Pto': 'PTO_IF',
    "Other Potential Network Upgrade": "OPNU",
    "Conditionally Assigned Network Upgrade": "CANU",
    "Canus": "CANU",
        "Escalated Cost And Time To Construct For Reliability Network Upgrade4": "RNU",
    "Escalated Cost And Time To Construct For Reliability Network Upgrade3": "RNU",
    "Local Off-Peak Network Upgrade": "LOPNU",
    'Total PTO_IF': 'PTO_IF',
 'Total RNU': 'RNU',
 'Total LDNU': 'LDNU',
 'Total OPNU' : 'OPNU',
 'Total CANU': 'CANU',
 'Total LOPNU': 'LOPNU',
 'Canu': 'CANU',
  
 'Total ADNU': 'ADNU',
  'Ptos Interconnect Ion Facilities' : 'PTO_IF',
  'Local Off- Peak Network Upgrade': 'LOPNU',
 'P Os Interconnection Facilities': 'PTO_IF',
  

 
}


#Step 3: Apply mapping and ffill type of upgrade column

if 'type_of_upgrade' in df.columns:
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: re.sub(r'\bupgrades\b', 'upgrade', x, flags=re.IGNORECASE).strip() if isinstance(x, str) else x
    )
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: mappings.get(x, x) if isinstance(x, str) else x
    )
    df['type_of_upgrade'] = df['type_of_upgrade'].ffill()  



import re
import pandas as pd


# build a boolean mask: True for rows where cost contains "(…)"
#has_paren = df['estimated_cost_x_1000'].astype(str).str.contains(r'\([^)]*\)')
# drop those rows
#df = df.loc[~has_paren].copy()

def drop_rows_with_star_in_costs(df, cost_cols):
    """
    Remove any row where any of the cost_cols contains a '*' character.
    """
    mask = pd.Series(False, index=df.index)
    for col in cost_cols:
        if col in df.columns:
            # star anywhere in the string
            mask = mask | df[col].astype(str).str.contains(r"\*", regex=True)
    # keep only rows without a star
    return df.loc[~mask].reset_index(drop=True)

def drop_rows_with_dash_in_time(df, time_col):
    """
    Remove any row where the time_col contains a dash '-' (e.g. '-').
    """
    if time_col in df.columns:
        mask = df[time_col].astype(str).str.contains(r"^-+$", regex=True)
        return df.loc[~mask].reset_index(drop=True)
    return df

def keep_second_entry_in_cells(df, columns):
    """
    For each column in columns, if the cell contains multiple space‑separated entries,
    keep only the second one.
    """
    for col in columns:
        if col not in df.columns:
            continue
        def pick_second(cell):
            parts = re.findall(r"[\d\.\$%,]+", str(cell))
            return parts[1] if len(parts) > 1 else (parts[0] if parts else cell)
        df[col] = df[col].apply(pick_second)
    return df

# ── Integration ──
# Place this just before your Step 7 clean_currency block:

# 1) drop any row where estimated or escalated cost has '*'
 

# 2) drop any row where estimated_time_to_construct is just a dash
df = drop_rows_with_dash_in_time(df, 'estimated_time_to_construct')

# 3) if multiple entries exist in a cell, keep only the second
 
# Now proceed with your clean_currency step…


    
# Step 3: Remove $ signs and convert to numeric

def clean_currency(value):
    """
    Cleans a string by explicitly removing $, *, (Note 2), and similar patterns,
    then converts it to a numeric value.
    """
    if isinstance(value, str):
        # Explicitly remove $, *, and any "(Note ...)"
        value = value.replace('$', '').replace('*', '')
        value = re.sub(r'\(Note \d+\)', '', value)  # Remove patterns like "(Note 2)"
         
    try:
        return value
    except ValueError:
        return pd.NA  # Return NaN for invalid entries
    

df['cost_allocation_factor'] = df['cost_allocation_factor'].apply(clean_currency)

import re

def clean_currency(value):
    """
    Cleans a string by explicitly removing $, *, (Note 2), and similar patterns,
    then converts it to a numeric value.
    """
    if isinstance(value, str):
        # Explicitly remove $, *, and any "(Note ...)"
        value = value.replace('$', '').replace('*', '')
        value = re.sub(r'\(Note \d+\)', '', value)  # Remove patterns like "(Note 2)"
        value = value.replace(',', '').strip()  # Remove commas and extra spaces
    try:
        return pd.to_numeric(value)
    except ValueError:
        return pd.NA  # Return NaN for invalid entries


# Clean the specific columns
for col in ['estimated_cost_x_1000', 'escalated_cost_x_1000', 'total_estimated_cost_x_1000_escalated', 'total_estimated_cost_x_1000', 'adnu_cost_rate_x_1000', 'adnu_cost_rate_x_1000_escalated']:
    if col in df.columns:
        df[col] = df[col].apply(clean_currency)

# Create Total rows
new_rows = []
columns_to_sum = ['estimated_cost_x_1000', 'escalated_cost_x_1000', 'total_estimated_cost_x_1000_escalated',   'adnu_cost_rate_x_1000_escalated']
columns_to_populate = ['cluster', 'req_deliverability', 'latitude', 'longitude', 'capacity', 'point_of_interconnection']

for q_id, group in df.groupby('q_id', as_index=False):
    unique_upgrades = group['type_of_upgrade'].dropna().unique()
    for upgrade in unique_upgrades:
        if pd.isna(upgrade):
            continue
        
        rows = group[group['type_of_upgrade'] == upgrade]

                # Check if a Total row already exists for this (q_id, upgrade)
        total_exists = group[
            (group['type_of_upgrade'] == upgrade) & (group['item'] == 'no')
        ].shape[0] > 0
        
        if total_exists:
             
            continue


 
        
        
        if not total_exists:
            # If only one row exists, duplicate it as the total row
            if len(rows) == 1:

                total_row = {col: '' for col in df.columns}  # Initialize all columns as empty strings

                # Populate the necessary fields
                total_row['q_id'] = q_id
                total_row['type_of_upgrade'] = f"Total {upgrade}"
                total_row['item'] = 'no'

                # Populate specified columns from the existing row
                first_row = rows.iloc[0]
                for col in columns_to_populate:
                    if col in df.columns:
                        total_row[col] = first_row[col]

                # Sum the numeric columns (single row, so it remains the same)
                for col in columns_to_sum:
                    if col in rows.columns:
                        total_row[col] = rows[col].sum()
                    else:
                        total_row[col] = 0  # Default to 0 if column is missing

                new_rows.append(total_row)



 
            
            # If multiple rows exist, sum numeric columns and create a total row
            elif len(rows) > 1:
                total_row = {col: '' for col in df.columns}  # Initialize all columns as empty strings

                # Populate the necessary fields
                total_row['q_id'] = q_id
                total_row['type_of_upgrade'] = f"Total {upgrade}"
                total_row['item'] = 'no'

                # Populate the specified columns from the first row in the group
                first_row = rows.iloc[0]
                for col in columns_to_populate:
                    if col in df.columns:
                        total_row[col] = first_row[col]

                # Sum the numeric columns
                for col in columns_to_sum:
                    if col in rows.columns:
                        total_row[col] = rows[col].sum()
                    else:
                        total_row[col] = 0  # Default to 0 if column is missing

                new_rows.append(total_row)
 



      

if new_rows:
    total_rows_df = pd.DataFrame(new_rows)
    for col in df.columns:
        if col not in total_rows_df.columns:
            total_rows_df[col] = None
    total_rows_df = total_rows_df[df.columns]
    df = pd.concat([df, total_rows_df], ignore_index=True)

df.reset_index(drop=True, inplace=True)

# Update 'item' column based on Total in type_of_upgrade or cost_allocation_factor
df['item'] = df.apply(
    lambda row: 'no' if (
        'Total' in str(row.get('type_of_upgrade', '')) or 
        'Total' in str(row.get('cost_allocation_factor', ''))
    ) else 'yes',
    axis=1
)


# Step 3: Move 'item' column next to 'type_of_upgrade'
if 'item' in df.columns and 'type_of_upgrade' in df.columns:
    cols = df.columns.tolist()
    item_index = cols.index('item')
    type_index = cols.index('type_of_upgrade')
    if item_index < type_index:
        cols.insert(type_index + 1, cols.pop(item_index))
    else:
        cols.insert(type_index + 1, cols.pop(item_index))
    df = df[cols]

# Step 9: Remove "Total" values from cost_allocation_factor if they appear in type_of_upgrade
if 'cost_allocation_factor' in df.columns and 'type_of_upgrade' in df.columns:
    df['cost_allocation_factor'] = df.apply(
        lambda row: None if (
            pd.notna(row['type_of_upgrade']) and 'Total' in str(row['type_of_upgrade'])
        ) else row.get('cost_allocation_factor'),
        axis=1
    )
    

if 'cost_allocation_factor' in df.columns and 'type_of_upgrade' in df.columns:
    df['cost_allocation_factor'] = df.apply(
        lambda row: None if 'Total' in str(row.get('cost_allocation_factor', '')) else row.get('cost_allocation_factor'),
        axis=1
    )



def clean_estimated_time(value):
    if isinstance(value, str):
        value = re.sub(r'(\d+(?:-\w+)*)\s+\w+.*$', r'\1', value, flags=re.IGNORECASE).strip()
    return value

if 'estimated_time_to_construct' in df.columns:
    df['estimated_time_to_construct'] = df['estimated_time_to_construct'].apply(clean_estimated_time)

if 'type_of_upgrade' in df.columns:
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: re.sub(r'\s*\(note \d+\)', '', x, flags=re.IGNORECASE).strip() if isinstance(x, str) else x
    )

 



if 'type_of_upgrade' in df.columns:
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: re.sub(r'\bupgrades\b', 'upgrade', x, flags=re.IGNORECASE).strip() if isinstance(x, str) else x
    )
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: mappings.get(x, x) if isinstance(x, str) else x
    )
    df['type_of_upgrade'] = df['type_of_upgrade'].ffill()

if 'upgrade' in df.columns and 'type_of_upgrade' in df.columns and 'q_id' in df.columns:
    df['upgrade'] = df.groupby(['q_id', 'type_of_upgrade'])['upgrade'].ffill()

if 'type_of_upgrade' in df.columns:
    previous_type_of_upgrade = None
    for i in range(len(df)):
        if df.at[i, 'type_of_upgrade'] == 'Total':
            if previous_type_of_upgrade is not None:
                df.at[i, 'type_of_upgrade'] = previous_type_of_upgrade
        else:
            previous_type_of_upgrade = df.at[i, 'type_of_upgrade']

numeric_columns = [
    'cost_allocation_factor',
    'estimated_cost_x_1000',
    'estimated_time_to_construct',
    'total_estimated_cost_x_1000_escalated',
    'adnu_cost_rate_x_1000',
    'escalated_cost_x_1000',
    'estimated_cost_x_1000_escalated_without_itcca',
    'adnu_cost_rate_x_1000_escalated'
]
non_numeric_columns = ['type_of_upgrade', 'upgrade', 'description']

for col in non_numeric_columns:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: 'None' if pd.isna(x) else x)

for col in numeric_columns:
    if col in df.columns:
        df[col] = df[col].replace('-', pd.NA)
        df[col] = df[col].fillna(0)

if 'original_order' in df.columns and 'q_id' in df.columns:
    df['original_order'] = df.index
    df = df.sort_values(by=['q_id', 'original_order'], ascending=[True, True])
    df = df.drop(columns=['original_order'])


def reorder_columns(df):
    """
    Reorders the columns of the DataFrame based on the specified order.

    Args:
        df (pd.DataFrame): The DataFrame to reorder.

    Returns:
        pd.DataFrame: The reordered DataFrame.
    """
    desired_order = [
        "q_id",
        "cluster",
        "req_deliverability",
        "latitude",
        "longitude",
        "capacity",
        "point_of_interconnection",
        "type_of_upgrade",
        "upgrade",
        "description",
        "cost_allocation_factor",
        "estimated_cost_x_1000",
        "escalated_cost_x_1000",
        "total_estimated_cost_x_1000",
        "total_estimated_cost_x_1000_escalated",
        "estimated_time_to_construct",
    ]

    # Start with desired columns that exist in the DataFrame
    existing_desired = [col for col in desired_order if col in df.columns]

    # Then add the remaining columns
    remaining = [col for col in df.columns if col not in existing_desired]

    # Combine the two lists
    new_order = existing_desired + remaining

    # Reorder the DataFrame
    df = df[new_order]

    return df        


df= reorder_columns(df)

# build a mask of exactly the rows you want to drop
mask = (
    (df['type_of_upgrade'] == 'PTO_IF') &
    (df['upgrade'] == 'None') &
    (df['description'] == 'None') &
    (df['cost_allocation_factor'] == 0) &
    (df['estimated_cost_x_1000'] == 0) &
    (df['escalated_cost_x_1000'] == 0) &
    (df['estimated_time_to_construct'] == 0)
)

 

# if you want to do it in-place instead:
df.drop(df[mask].index, inplace=True)

# Save itemized and totals separately
if 'item' in df.columns:
    itemized_df = df[df['item'] == 'yes']
    itemized_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/02_intermediate/costs_phase_2_cluster_3_style_new_itemized_addendums.csv', index=False)

    totals_columns = ['upgrade', 'description', 'cost_allocation_factor', 'estimated_time_to_construct']
    totals_df = df[df['item'] == 'no'].drop(columns=totals_columns, errors='ignore')
    totals_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/02_intermediate/costs_phase_2_cluster_3_style_new_total_addendums.csv', index=False)

print(f"Itemized rows saved to 'costs_phase_2_cluster_3_style_new_itemized.csv'.")
print(f"Filtered Total rows saved to 'costs_phase_2_cluster_3_style_new_total.csv'.")


if 'type_of_upgrade' in df.columns:
    print(df['type_of_upgrade'].unique())

if 'q_id' in df.columns:
    print(df['q_id'].unique())

if 'cluster' in df.columns:
    print(df['cluster'].unique())


Itemized rows saved to 'costs_phase_2_cluster_3_style_new_itemized.csv'.
Filtered Total rows saved to 'costs_phase_2_cluster_3_style_new_total.csv'.
['PTO_IF' 'RNU' 'LDNU']
['643W']
[3]


# Merge- Complete replace

In [16]:
import pandas as pd
import numpy as np

def load_data(file_path, char_columns):
    """
    Load a CSV file and ensure specific columns are treated as character.
    """
    available_columns = pd.read_csv(file_path, nrows=0).columns
    char_cols = [c for c in char_columns if c in available_columns]
    return pd.read_csv(
        file_path,
        dtype={c: str for c in char_cols},
        na_values=[], 
        keep_default_na=False
    )

def save_data(df, file_path, char_columns):
    """
    Save a dataframe to CSV, forcing certain columns to string.
    """
    for col in char_columns:
        if col in df.columns:
            df[col] = df[col].astype(str)
    df.to_csv(file_path, index=False)

def merge_with_addendums(itemized, itemized_addendums, total, total_addendums):
    # mark originals & keep row order
    itemized['original'] = "yes"
    total['original']    = "yes"
    itemized['row_order'] = itemized.index
    total['row_order']    = total.index

    # ensure numeric q_id
 

    conditional_columns = [
        "req_deliverability","latitude","longitude",
        "capacity","point_of_interconnection"
    ]

    # --- ITEMIZED: replace only matching (q_id, type_of_upgrade) blocks ---
    updated_itemized_rows = []
    # iterate over each unique (q_id, type_of_upgrade) in the addendums
    for q, t in itemized_addendums[['q_id','type_of_upgrade']].drop_duplicates().itertuples(index=False):
        adds = itemized_addendums[
            (itemized_addendums['q_id'] == q) &
            (itemized_addendums['type_of_upgrade'] == t)
        ].reset_index(drop=True)
        orig = itemized[
            (itemized['q_id'] == q) &
            (itemized['type_of_upgrade'] == t)
        ].reset_index(drop=True)

        # combine conditional columns
        for col in conditional_columns:
            if col in adds.columns and col in orig.columns:
                adds[col] = (
                    adds[col].replace("", pd.NA)
                              .combine_first(orig[col])
                              .fillna("")
                )

        # carry over or pad row_order
        if 'row_order' in orig:
            ro = orig['row_order'].tolist()
            if len(ro) < len(adds):
                ro += [pd.NA] * (len(adds) - len(ro))
        else:
            ro = [pd.NA] * len(adds)

        adds = adds.assign(original="no", row_order=ro[:len(adds)])

        # drop only those matching (q_id, type_of_upgrade) from the master
        itemized = itemized[
            ~((itemized['q_id'] == q) & (itemized['type_of_upgrade'] == t))
        ]

        updated_itemized_rows.append(adds)

    # stitch back untouched originals + updated blocks
    updated_itemized = pd.concat(
        [itemized] + updated_itemized_rows,
        ignore_index=True
    ) if updated_itemized_rows else itemized.copy()

    updated_itemized['row_order'] = updated_itemized['row_order'].fillna(-1).astype(int)
    updated_itemized = (
        updated_itemized
        .sort_values('row_order')
        .drop(columns='row_order')
        .reset_index(drop=True)
    )

    # --- TOTAL: per type_of_upgrade (unchanged logic) ---
    updated_total_rows = []
    for q in total_addendums['q_id'].unique():
        for t in total_addendums['type_of_upgrade'].unique():
            adds = total_addendums[
                (total_addendums['q_id']==q)&
                (total_addendums['type_of_upgrade']==t)
            ].reset_index(drop=True)
            if adds.empty:
                continue

            mask = (total['q_id']==q)&(total['type_of_upgrade']==t)
            orig = total[mask].reset_index(drop=True)
            if orig.empty:
                orig = pd.DataFrame({'row_order':[pd.NA]*len(adds)}, index=adds.index)

            # align lengths
            if len(adds) > len(orig):
                extra = pd.DataFrame({c: pd.NA for c in orig.columns},
                                     index=range(len(adds)-len(orig)))
                orig = pd.concat([orig, extra], ignore_index=True)
            elif len(adds) < len(orig):
                orig = orig.iloc[:len(adds)].reset_index(drop=True)

            for col in conditional_columns:
                if col in adds.columns and col in orig.columns:
                    adds[col] = (
                        adds[col].replace("", pd.NA)
                                  .combine_first(orig[col])
                                  .fillna("")
                    )

            total.loc[mask, 'original'] = "no"
            updated_total_rows.append(
                adds.assign(original="no", row_order=orig['row_order'].tolist()[:len(adds)])
            )
            total = total[~mask]

    updated_total = pd.concat([total] + updated_total_rows, ignore_index=True) \
                    if updated_total_rows else total.copy()
    updated_total['row_order'] = updated_total['row_order'].fillna(-1).astype(int)
    updated_total = (
        updated_total
        .sort_values('row_order')
        .drop(columns='row_order')
        .reset_index(drop=True)
    )

    # move 'original' to end
    def move_last(df):
        cols = [c for c in df.columns if c!='original'] + ['original']
        return df[cols]

    return move_last(updated_itemized), move_last(updated_total)


# ── main script ──

char_columns = [
    "req_deliverability","point_of_interconnection","type_of_upgrade",
    "upgrade","description","estimated_time_to_construct","original","item"
]

itemized = load_data(
    "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/"
    "04_intermediate_scraped_data/phase_2_cost_data/"
    "Cluster 3/02_intermediate/"
    "costs_phase_2_cluster_3_style_new_itemized.csv",
    char_columns
)
itemized_addendums = load_data(
    "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/"
    "04_intermediate_scraped_data/phase_2_cost_data/"
    "Cluster 3/02_intermediate/"
    "costs_phase_2_cluster_3_style_new_itemized_addendums.csv",
    char_columns
)
total = load_data(
    "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/"
    "04_intermediate_scraped_data/phase_2_cost_data/"
    "Cluster 3/02_intermediate/"
    "costs_phase_2_cluster_3_style_new_total.csv",
    char_columns
)
total_addendums = load_data(
    "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/"
    "04_intermediate_scraped_data/phase_2_cost_data/"
    "Cluster 3/02_intermediate/"
    "costs_phase_2_cluster_3_style_new_total_addendums.csv",
    char_columns
)

updated_itemized, updated_total = merge_with_addendums(
    itemized, itemized_addendums, total, total_addendums
)

# drop unwanted columns
to_drop = [
    "upgrade_classification","estimated","caiso_queue",
    "project_type","dependent_system_upgrade"
]
updated_itemized = updated_itemized.drop(columns=[c for c in to_drop if c in updated_itemized], errors='ignore')
updated_total   = updated_total.drop(columns=[c for c in to_drop if c in updated_total],   errors='ignore')

# fill & sort
fill_cols = [
    "point_of_interconnection","latitude","longitude",
    "req_deliverability","capacity"
]
for df in (updated_itemized, updated_total):
    for c in fill_cols:
        df[c] = df[c].replace('', np.nan)
    df.sort_values('q_id', kind='stable', inplace=True)
    df.reset_index(drop=True, inplace=True)
    for c in fill_cols:
        df[c] = df.groupby('q_id')[c].apply(lambda g: g.ffill().bfill()).reset_index(drop=True)
    for c in fill_cols:
        df[c].replace(np.nan, '', inplace=True)

# save
save_data(
    updated_itemized,
    "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/"
    "04_intermediate_scraped_data/phase_2_cost_data/"
    "Cluster 3/01_clean/"
    "costs_phase_2_cluster_3_style_new_itemized_updated.csv",
    char_columns
)
save_data(
    updated_total,
    "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/"
    "04_intermediate_scraped_data/phase_2_cost_data/"
    "Cluster 3/01_clean/"
    "costs_phase_2_cluster_3_style_new_total_updated.csv",
    char_columns
)


/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_21229/589116160.py:86: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  updated_itemized['row_order'] = updated_itemized['row_order'].fillna(-1).astype(int)
/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_21229/589116160.py:205: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] = df[c].replace('', np.nan)
/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_21229/589116160.py:211: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through cha

# Checking Scraped Data

# Orignals only

# Trying to compare the total cost across all types of upgrade as that is given in the pdfs

In [10]:
import pandas as pd

# ---------------------- Configuration ---------------------- #

ITEMIZED_CSV_PATH       = '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/02_intermediate/costs_phase_2_cluster_3_style_new_itemized.csv'
TOTALS_CSV_PATH         = '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/02_intermediate/costs_phase_2_cluster_3_style_new_total.csv'
AGGREGATE_CSV_PATH      = '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/02_intermediate/costs_phase_2_cluster_3_style_new_aggregate.csv'

TOTALS_ESTIMATED_COLUMN = 'estimated_cost_x_1000'
TOTALS_ESCALATED_COLUMN = 'escalated_cost_x_1000'

REQUIRED_UPGRADES       = ['PTO_IF', 'RNU', 'LDNU', 'ADNU']

MISMATCHES_CSV_PATH     = '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/mismatches.csv'

# ---------------------- Load Data ---------------------- #

itemized_df = pd.read_csv(ITEMIZED_CSV_PATH, dtype={'type_of_upgrade': str})
totals_df   = pd.read_csv(TOTALS_CSV_PATH, dtype={'type_of_upgrade': str})
agg_df      = pd.read_csv(AGGREGATE_CSV_PATH, dtype=str)

# ---------------------- Clean aggregate costs ---------------------- #

# Remove $ and commas, then convert to float
for col in [TOTALS_ESTIMATED_COLUMN, TOTALS_ESCALATED_COLUMN]:
    agg_df[col] = (
        agg_df[col]
        .str.replace(r'[\$,]', '', regex=True)
        .astype(float)
        .fillna(0.0)
    )

# ---------------------- Build aggregate lookup ---------------------- #

agg_grouped = (
    agg_df
    .groupby('q_id', as_index=False)
    .agg({
        TOTALS_ESTIMATED_COLUMN: 'sum',
        TOTALS_ESCALATED_COLUMN: 'sum'
    })
)

# create lookup
agg_lookup = agg_grouped.set_index('q_id').to_dict(orient='index')
agg_qids   = set(agg_grouped['q_id'])

# ---------------------- Numeric convert itemized ---------------------- #

for col in ['estimated_cost_x_1000','escalated_cost_x_1000']:
    itemized_df[col] = (
        itemized_df[col]
        .astype(str)
        .str.replace(r'[\$,]', '', regex=True)
        .astype(float)
        .fillna(0.0)
    )

# ---------------------- Check missing upgrades for  Q_ids ---------------------- #

# ---------------------- Check missing upgrades in totals_df (unconditionally) ---------------------- #

print("=== Missing required upgrades in totals dataset ===")
missing = []
for q in sorted(totals_df['q_id'].unique()):
    ups = (
        totals_df
        .loc[totals_df['q_id'] == q, 'type_of_upgrade']
        .dropna()
        .unique()
        .tolist()
    )
    miss = [u for u in REQUIRED_UPGRADES if u not in ups]
    if miss:
        missing.append((q, miss))

if missing:
    for q, miss in missing:
        print(f"Q_id {q} missing: {miss}")
else:
    print("None — every Q_id has all required upgrades in totals_df.")


# ---------------------- Check duplicate upgrades in totals dataset ---------------------- #

print("\n=== Duplicate upgrades in totals dataset ===")
dups = []
for q, group in totals_df.groupby('q_id'):
    dup_types = group['type_of_upgrade'][group['type_of_upgrade'].duplicated()].unique().tolist()
    if dup_types:
        dups.append((q, dup_types))

if dups:
    for q, dup in dups:
        print(f"Q_id {q} duplicates: {dup}")
else:
    print("No duplicates found in totals dataset.")

# ---------------------- Compute per-q_id itemized total ---------------------- #

itemized_totals = (
    itemized_df[itemized_df['q_id'].isin(agg_qids)]
    .groupby('q_id', as_index=False)
    .agg({
        'estimated_cost_x_1000':'sum',
        'escalated_cost_x_1000':'sum'
    })
)

itemized_totals['itemized_total'] = itemized_totals.apply(
    lambda r: r['estimated_cost_x_1000'] if r['estimated_cost_x_1000']>0 else r['escalated_cost_x_1000'],
    axis=1
)

# ---------------------- Compare against aggregate totals ---------------------- #

mismatches = []
for _, row in itemized_totals.iterrows():
    q = row['q_id']
    it = row['itemized_total']
    av = agg_lookup[q][TOTALS_ESTIMATED_COLUMN] if agg_lookup[q][TOTALS_ESTIMATED_COLUMN]>0 else agg_lookup[q][TOTALS_ESCALATED_COLUMN]
    # skip both zero
    if it==0 and av==0:
        continue
    if abs(it - av) > 1e-6:
        mismatches.append({
            'q_id': q,
            'itemized_total': it,
            'aggregate_total': av,
            'difference': it - av
        })

mismatches_df = pd.DataFrame(mismatches)

# ---------------------- Report & Save ---------------------- #

if mismatches_df.empty:
    print("\n✅ All itemized sums match the aggregate totals for Q_ids in aggregate.")
else:
    print(f"\n⚠️  Found {len(mismatches_df)} mismatches:")
    print(mismatches_df)

mismatches_df.to_csv(MISMATCHES_CSV_PATH, index=False)
print(f"\nMismatches written to {MISMATCHES_CSV_PATH}")


=== Missing required upgrades in totals dataset ===
Q_id 643I missing: ['RNU', 'ADNU']
Q_id 643W missing: ['ADNU']

=== Duplicate upgrades in totals dataset ===
No duplicates found in totals dataset.

✅ All itemized sums match the aggregate totals for Q_ids in aggregate.

Mismatches written to /Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/mismatches.csv


# addendums

In [9]:
import pandas as pd

# ---------------------- Configuration ---------------------- #

ITEMIZED_CSV_PATH       = '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/02_intermediate/costs_phase_2_cluster_3_style_new_itemized_addendums.csv'
TOTALS_CSV_PATH         = '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/02_intermediate/costs_phase_2_cluster_3_style_new_total_addendums.csv'
AGGREGATE_CSV_PATH      = '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/02_intermediate/costs_phase_2_cluster_3_style_new_aggregate_addendums.csv'

TOTALS_ESTIMATED_COLUMN = 'estimated_cost_x_1000'
TOTALS_ESCALATED_COLUMN = 'escalated_cost_x_1000'

REQUIRED_UPGRADES       = ['PTO_IF', 'RNU', 'LDNU', 'ADNU']

MISMATCHES_CSV_PATH     = '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/mismatches.csv'

# ---------------------- Load Data ---------------------- #

itemized_df = pd.read_csv(ITEMIZED_CSV_PATH, dtype={'type_of_upgrade': str})
totals_df   = pd.read_csv(TOTALS_CSV_PATH, dtype={'type_of_upgrade': str})
agg_df      = pd.read_csv(AGGREGATE_CSV_PATH, dtype=str)

# ---------------------- Clean aggregate costs ---------------------- #

# Remove $ and commas, then convert to float
for col in [TOTALS_ESTIMATED_COLUMN, TOTALS_ESCALATED_COLUMN]:
    agg_df[col] = (
        agg_df[col]
        .str.replace(r'[\$,]', '', regex=True)
        .astype(float)
        .fillna(0.0)
    )

# ---------------------- Build aggregate lookup ---------------------- #

agg_grouped = (
    agg_df
    .groupby('q_id', as_index=False)
    .agg({
        TOTALS_ESTIMATED_COLUMN: 'sum',
        TOTALS_ESCALATED_COLUMN: 'sum'
    })
)

# create lookup
agg_lookup = agg_grouped.set_index('q_id').to_dict(orient='index')
agg_qids   = set(agg_grouped['q_id'])

# ---------------------- Numeric convert itemized ---------------------- #

for col in ['estimated_cost_x_1000','escalated_cost_x_1000']:
    itemized_df[col] = (
        itemized_df[col]
        .astype(str)
        .str.replace(r'[\$,]', '', regex=True)
        .astype(float)
        .fillna(0.0)
    )

# ---------------------- Check missing upgrades for  Q_ids ---------------------- #

# ---------------------- Check missing upgrades in totals_df (unconditionally) ---------------------- #

print("=== Missing required upgrades in totals dataset ===")
missing = []
for q in sorted(totals_df['q_id'].unique()):
    ups = (
        totals_df
        .loc[totals_df['q_id'] == q, 'type_of_upgrade']
        .dropna()
        .unique()
        .tolist()
    )
    miss = [u for u in REQUIRED_UPGRADES if u not in ups]
    if miss:
        missing.append((q, miss))

if missing:
    for q, miss in missing:
        print(f"Q_id {q} missing: {miss}")
else:
    print("None — every Q_id has all required upgrades in totals_df.")


# ---------------------- Check duplicate upgrades in totals dataset ---------------------- #

print("\n=== Duplicate upgrades in totals dataset ===")
dups = []
for q, group in totals_df.groupby('q_id'):
    dup_types = group['type_of_upgrade'][group['type_of_upgrade'].duplicated()].unique().tolist()
    if dup_types:
        dups.append((q, dup_types))

if dups:
    for q, dup in dups:
        print(f"Q_id {q} duplicates: {dup}")
else:
    print("No duplicates found in totals dataset.")

# ---------------------- Compute per-q_id itemized total ---------------------- #

itemized_totals = (
    itemized_df[itemized_df['q_id'].isin(agg_qids)]
    .groupby('q_id', as_index=False)
    .agg({
        'estimated_cost_x_1000':'sum',
        'escalated_cost_x_1000':'sum'
    })
)

itemized_totals['itemized_total'] = itemized_totals.apply(
    lambda r: r['estimated_cost_x_1000'] if r['estimated_cost_x_1000']>0 else r['escalated_cost_x_1000'],
    axis=1
)

# ---------------------- Compare against aggregate totals ---------------------- #

mismatches = []
for _, row in itemized_totals.iterrows():
    q = row['q_id']
    it = row['itemized_total']
    av = agg_lookup[q][TOTALS_ESTIMATED_COLUMN] if agg_lookup[q][TOTALS_ESTIMATED_COLUMN]>0 else agg_lookup[q][TOTALS_ESCALATED_COLUMN]
    # skip both zero
    if it==0 and av==0:
        continue
    if abs(it - av) > 1e-6:
        mismatches.append({
            'q_id': q,
            'itemized_total': it,
            'aggregate_total': av,
            'difference': it - av
        })

mismatches_df = pd.DataFrame(mismatches)

# ---------------------- Report & Save ---------------------- #

if mismatches_df.empty:
    print("\n✅ All itemized sums match the aggregate totals for Q_ids in aggregate.")
else:
    print(f"\n⚠️  Found {len(mismatches_df)} mismatches:")
    print(mismatches_df)

mismatches_df.to_csv(MISMATCHES_CSV_PATH, index=False)
print(f"\nMismatches written to {MISMATCHES_CSV_PATH}")


=== Missing required upgrades in totals dataset ===
Q_id 643W missing: ['ADNU']

=== Duplicate upgrades in totals dataset ===
No duplicates found in totals dataset.

⚠️  Found 1 mismatches:
   q_id  itemized_total  aggregate_total  difference
0  643W         53104.0          53105.0        -1.0

Mismatches written to /Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 3/mismatches.csv


# Final Data